In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors
import csv
import numpy as np
import pandas as pd
import cobra

In [2]:
Annotation = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Annotation', index_col=0)
Annotation.index = Annotation.index.map(str)
Annotation = Annotation.fillna('')
Transcriptomics = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Transcriptomics', header=[0,1,2,3], index_col=0)
Transcriptomics.index = Transcriptomics.index.map(str)
Proteomics = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Proteomics', header=[0,1,2], index_col=0)
Proteomics.index = Proteomics.index.map(str)
Fitness = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Fitness', index_col=0)
Fitness.index = Fitness.index.map(str)

In [3]:
def background_gradient(s, cmap='seismic', text_color_threshold=0.408):
    lim = max(abs(s.min().min()),abs(s.max().max()))
    rng = 2.0*lim
    norm = colors.Normalize(-lim - (rng * 0.2), lim + (rng * 0.2))
    rgbas = plt.cm.get_cmap(cmap)(norm(s.values))
    def relative_luminance(rgba):
        r, g, b = (x / 12.92 if x <= 0.03928 else ((x + 0.055) / 1.055 ** 2.4) for x in rgba[:3])
        return 0.2126 * r + 0.7152 * g + 0.0722 * b
    def css(rgba):
        dark = relative_luminance(rgba) < text_color_threshold
        text_color = '#f1f1f1' if dark else '#000000'
        return 'background-color: {b};color: {c};'.format(b=colors.rgb2hex(rgba), c=text_color)

    if s.ndim == 1:
        return [css(rgba) for rgba in rgbas]
    else:
        return pd.DataFrame([[css(rgba) for rgba in row] for row in rgbas], index=s.index, columns=s.columns)

def Show_Data(x):
    display(Transcriptomics.loc[x].style.background_gradient(cmap='Reds', low=0.2, high=0.2, axis=None))
    temp = [y for y in x if y in Proteomics.index]
    display(Proteomics.loc[temp].style.background_gradient(cmap='Reds', low=0.2, high=0.2, axis=None))
    temp = [y for y in x if y in Fitness.index]
    display(Fitness.loc[temp].style.apply(background_gradient, cmap='seismic', axis=None))
    return;

In [4]:
model = cobra.io.load_json_model("../1.Manual_curation/IFO0880_GPR_1f.json")

In [5]:
eco = cobra.io.load_json_model('../../Data/BiGG_Models/iML1515.json')
sce = cobra.io.load_json_model('../../Data/BiGG_Models/iMM904.json')
hsa = cobra.io.load_json_model('../../Data/BiGG_Models/RECON1.json')
hsa2 = cobra.io.load_json_model('../../Data/BiGG_Models/Recon3D.json')
ptri = cobra.io.load_json_model('../../Data/BiGG_Models/iLB1027_lipid.json')

In [6]:
Reaction_manual = ['EX_co2_e','EX_h_e','EX_h2o_e','EX_nh4_e','EX_o2_e','EX_pi_e','EX_so4_e','EX_glc__D_e',
                   'H2Ot','H2Oter','H2Otg','H2Otm','H2Otn','H2Otp','H2Otv',
                   'CO2t','CO2ter','CO2tg','CO2tm','CO2tn','CO2tp','CO2tv',
                   'O2t','O2ter','O2tm','O2tn','O2tp']
print([x for x in Reaction_manual if x in model.reactions])
print([(r.id, r.gene_reaction_rule)  for r in sce.reactions if r.id in Reaction_manual and r.gene_reaction_rule])
print([(r.id, r.gene_reaction_rule)  for r in hsa.reactions if r.id in Reaction_manual and r.gene_reaction_rule])

['H2Ot', 'H2Otm', 'H2Otp']
[('H2Ot', 'YPR192W or YLL052C')]
[('H2Ot', '6523_AT1'), ('H2Otm', '343_AT1')]


In [7]:
for x in ['H2Ot', 'H2Otm', 'H2Otp']:
    print(x,model.reactions.get_by_id(x).gene_reaction_rule)

H2Ot 13986 or 13987 or 9014 or 9015
H2Otm 13986 or 13987 or 9014 or 9015
H2Otp 13986


In [8]:
#temp = Annotation.index[Annotation['Combined Annotations'].str.contains('aquaporin')]
temp = Annotation.index[Annotation['Human Blast'].str.contains('AQP', na=False)]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
9014,"K09885: AQPF; aquaporin rerated protein, other...",,AQY1,"AQP4,AQP1,AQP8",AQY1,AQP4,Not Essential,"plas 18, mito 4, E.R. 2, cyto_mito 2",EKV*
9015,"K09885: AQPF; aquaporin rerated protein, other...",A,AQY1,"AQP4,AQP1,AQP8",AQY1,AQP1,Not Essential,plas 27,SAS*
13100,"HMMPfam:Major intrinsic protein:PF00230,PRINTS...",,,,YFL054C,AQP10,Not Essential,"plas 21, E.R. 4",LKA*
13986,K09866: AQP4; aquaporin-4,,AQY1,"AQP4,AQP1,AQP8",AQY1,AQP4,Not Essential,"plas 23, E.R. 3",SPV*
13987,"K09885: AQPF; aquaporin rerated protein, other...",,AQY1,"AQP4,AQP1,AQP8",AQY1,AQP1,Not Essential,"plas 24, E.R. 2",RFA*
15470,K03441: GLP-F; aquaglyceroporin related protei...,,"YFL054C,FPS1","AQP3,AQP9",YFL054C,AQP3,Not Essential,"plas 17, nucl 6, mito 1, cyto 1, E.R. 1, vacu ...",WRQ*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
9014,-0.0989641,-0.0117056,-0.217921,-0.106086,-0.110371,0.18789,-0.442884,-0.248592,-0.165871,-0.342108,-0.131261
9015,-0.0247873,-0.200422,-0.0550922,-0.197392,0.177297,-0.108184,0.511344,-0.225037,-0.229903,-0.0266842,-0.165627
13100,0.110137,0.219541,0.237074,0.153246,0.575455,0.53134,-0.0581222,-0.293301,-0.353711,-0.27729,0.269053
13986,0.266698,0.148965,0.291259,0.155225,0.191408,0.108453,0.340606,0.278426,0.062012,0.0155407,0.383375
13987,0.171695,0.299718,-0.0115636,0.288009,-0.0296966,0.106285,0.263578,-0.712035,-0.210913,-0.0287829,1.6117
15470,0.00318651,0.0512761,-0.167947,0.0916783,0.0498898,-0.185868,0.158407,-0.399259,0.128998,-0.140828,-0.115934


In [9]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

H2Ot h2o_e <=> h2o_c 13986 or 13987 or 9014 or 9015
H2Otm h2o_c <=> h2o_m 13986 or 13987 or 9014 or 9015

H2Ot h2o_e <=> h2o_c 13986 or 13987 or 9014 or 9015
H2Otm h2o_c <=> h2o_m 13986 or 13987 or 9014 or 9015

13100 no reactions

H2Ot h2o_e <=> h2o_c 13986 or 13987 or 9014 or 9015
H2Otm h2o_c <=> h2o_m 13986 or 13987 or 9014 or 9015
H2Otp h2o_c <=> h2o_x 13986

H2Ot h2o_e <=> h2o_c 13986 or 13987 or 9014 or 9015
H2Otm h2o_c <=> h2o_m 13986 or 13987 or 9014 or 9015

GLYCt glyc_c <=> glyc_e 15470
L_LACtcm lac__L_c --> lac__L_m 15470
UREAtm urea_c <=> urea_m 15470



In [10]:
# H2Otm in Recon is by AQP8, leave 9014 since it has mito in WolfPSort
model.reactions.get_by_id('H2Otm').gene_reaction_rule = '9014'
model.reactions.get_by_id('H2Otp').gene_reaction_rule = ''
# 15470 is Aquaglyceroporin, remove L_LACtcm
# 13100 is also Aquaglyceroporin
model.reactions.get_by_id('GLYCt').gene_reaction_rule = '13100 or 15470'
r = hsa2.reactions.get_by_id('UREAt').copy()
r.gene_reaction_rule = '13100 or 15470'
model.add_reactions([r])

In [11]:
for x in Reaction_manual:
    if x not in model.reactions:
        if x in sce.reactions:
            r = sce.reactions.get_by_id(x)
            print(r.id, r.reaction, r.gene_reaction_rule)
        elif x in hsa.reactions:
            r = hsa.reactions.get_by_id(x)
            print(r.id, r.reaction, r.gene_reaction_rule)
        else:
            print(x, 'not found in sce or hsa')

EX_co2_e co2_e -->  
EX_h_e h_e <=>  
EX_h2o_e h2o_e <=>  
EX_nh4_e nh4_e <=>  
EX_o2_e o2_e <=>  
EX_pi_e pi_e <=>  
EX_so4_e so4_e <=>  
EX_glc__D_e glc__D_e <=>  
H2Oter h2o_c <=> h2o_r 
H2Otg h2o_c <=> h2o_g 
H2Otn h2o_n <=> h2o_c 
H2Otv h2o_c <=> h2o_v 
CO2t co2_e <=> co2_c 
CO2ter co2_c <=> co2_r 
CO2tg co2_c <=> co2_g 
CO2tm co2_c <=> co2_m 
CO2tn co2_n <=> co2_c 
CO2tp co2_c <=> co2_x 
CO2tv co2_c <=> co2_v 
O2t o2_e <=> o2_c 
O2ter o2_c <=> o2_r 
O2tm o2_c <=> o2_m 
O2tn o2_c <=> o2_n 
O2tp o2_c <=> o2_x 


In [12]:
for x in Reaction_manual:
    if x not in model.reactions:
        if x in sce.reactions:
            model.add_reaction(sce.reactions.get_by_id(x).copy())
        elif x in hsa.reactions:
            model.add_reaction(hsa.reactions.get_by_id(x).copy())
        else:
            print(x, 'not found in sce or hsa')

In [13]:
r = sce.reactions.get_by_id('ATPM').copy()
model.add_reactions([r])
r

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x0102b9d6b00
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP C10H12N5O13P3 + H2O H2O --> ADP C10H12N5O10P2 + H+ + Phosphate
GPR,
Lower bound,1.0
Upper bound,1.0


In [14]:
Biomass = sce.reactions.get_by_id('BIOMASS_SC5_notrace').copy()
Biomass.id = 'BIOMASS_RT'
Biomass.name = 'Biomass'
Biomass

Reaction identifier,BIOMASS_RT
Name,Biomass
Memory address,0x0102ba6abe0
Stoichiometry,1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024 dgmp... 1.1348 1 3 beta D Glucan C6H10O5 + 0.4588 L-Alanine + 0.046 AMP C10H12N5O7P + 0.1607 L-Arginine + 0.1017 L-Asparagine + 0.2975 L-Aspartate + 59.276 ATP C10H12N5O13P3 + 0.0447 CMP C9H12N3O8P + 0.006...
GPR,
Lower bound,0.0
Upper bound,999999.0


In [15]:
for m in Biomass.metabolites:
    if not m.id in model.metabolites:
        if not 'SC' in m.id:
            print(m.id)
        elif m.id.replace('_SC','_RT') in model.metabolites:
            print(m.id, m.id.replace('_SC','_RT'))
        elif m.id.replace('_SC_c','_RT_r') in model.metabolites:
            print(m.id, m.id.replace('_SC_c','_RT_r'))

mannan_c
pa_SC_c pa_RT_r
pc_SC_c pc_RT_r
pe_SC_c pe_RT_r
ps_SC_c ps_RT_r
ptd1ino_SC_c ptd1ino_RT_r
triglyc_SC_c triglyc_RT_r


In [16]:
for r in sorted(model.metabolites.get_by_id('mannan_r').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dolmanp_r').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

DOLPMMer: dolmanp_r --> dolp_r + h_r + mannan_r 12121 or 13840 or 8768 or (13840 and 8768)

DOLPMMer: dolmanp_r --> dolp_r + h_r + mannan_r 12121 or 13840 or 8768 or (13840 and 8768)
DOLPMTcer: dolp_c + gdpmann_c --> dolmanp_r + gdp_c 13484


In [17]:
model.reactions.get_by_id('DOLPMMer').gene_reaction_rule = '12121 or 13840 or 8768'
model.add_reactions([Biomass])
Biomass.add_metabolites({'mannan_c': -Biomass.get_coefficient('mannan_c'),
                         'mannan_r': Biomass.get_coefficient('mannan_c')})
Biomass.add_metabolites({'pa_SC_c': -Biomass.get_coefficient('pa_SC_c'),
                         'pa_RT_r': Biomass.get_coefficient('pa_SC_c')})
Biomass.add_metabolites({'pc_SC_c': -Biomass.get_coefficient('pc_SC_c'),
                         'pc_RT_r': Biomass.get_coefficient('pc_SC_c')})
Biomass.add_metabolites({'pe_SC_c': -Biomass.get_coefficient('pe_SC_c'),
                         'pe_RT_r': Biomass.get_coefficient('pe_SC_c')})
Biomass.add_metabolites({'ps_SC_c': -Biomass.get_coefficient('ps_SC_c'),
                         'ps_RT_r': Biomass.get_coefficient('ps_SC_c')})
Biomass.add_metabolites({'ptd1ino_SC_c': -Biomass.get_coefficient('ptd1ino_SC_c'),
                         'ptd1ino_RT_r': Biomass.get_coefficient('ptd1ino_SC_c')})
Biomass.add_metabolites({'triglyc_SC_c': -Biomass.get_coefficient('triglyc_SC_c'),
                         'triglyc_RT_r': Biomass.get_coefficient('triglyc_SC_c')})

In [18]:
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)
model.remove_metabolites([x for x in model.metabolites if not x.reactions])

mannan_c
pa_SC_c
pc_SC_c
pe_SC_c
ps_SC_c
ptd1ino_SC_c
triglyc_SC_c


In [19]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.check_mass_balance() and not r.boundary:
        if sum(abs(x) for x in r.check_mass_balance().values()) > 1e-6:
            print(r, r.check_mass_balance())

BIOMASS_RT: 1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024 dgmp_c + 0.0036 dtmp_c + 0.0007 ergst_c + 0.1054 gln__L_c + 0.3018 glu__L_c + 0.2904 gly_c + 0.5185 glycogen_c + 0.046 gmp_c + 59.276 h2o_c + 0.0663 his__L_c + 0.1927 ile__L_c + 0.2964 leu__L_c + 0.2862 lys__L_c + 0.8079 mannan_r + 0.0507 met__L_c + 6e-06 pa_RT_r + 6e-05 pc_RT_r + 4.5e-05 pe_RT_r + 0.1339 phe__L_c + 0.1647 pro__L_c + 1.7e-05 ps_RT_r + 5.3e-05 ptd1ino_RT_r + 0.00099 ribflv_c + 0.1854 ser__L_c + 0.02 so4_c + 0.1914 thr__L_c + 0.0234 tre_c + 6.6e-05 triglyc_RT_r + 0.0284 trp__L_c + 0.102 tyr__L_c + 0.0599 ump_c + 0.2646 val__L_c + 0.0015 zymst_c --> 59.276 adp_c + 58.70001 h_c + 59.305 pi_c {'charge': -0.01618999999994411, 'C': -35.94203000000009, 'H': -65.57310999999986, 'O': -23.6762400000002, 'N': -5.596460000000039, 'P': -0.1977000000000017, 'S': -0.07730000000000001

## Fatty acid transport reactions for lipid biosynthesis and mobilization

In [20]:
# Mito fatty acid transport
for r in sorted(model.metabolites.get_by_id('crn_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('crn_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('crn_x').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('crn_r').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

ARACHCPT1: arachcoa_c + crn_c --> arachcrn_c + coa_c 9315
C160CPT1: crn_c + pmtcoa_c --> coa_c + pmtcrn_c 9315
C161CPT1: crn_c + hdcoa_c --> coa_c + hdcecrn_c 9315
C161CPT12: crn_c + hdd2coa_c --> coa_c + hdd2crn_c 9315
C181CPT1: crn_c + odecoa_c --> coa_c + odecrn_c 9315
CLPNDCPT1: clpndcoa_c + crn_c --> clpndcrn_c + coa_c 9315
CRNCARtm: acrn_c + crn_m --> acrn_m + crn_c 9331
CRNtim: crn_m --> crn_c 9331
CSNATr: accoa_c + crn_c <=> acrn_c + coa_c 13580 or 14245
DCSPTN1CPT1: crn_c + dcsptn1coa_c --> coa_c + dcsptn1crn_c 9315
ELAIDCPT1: crn_c + od2coa_c --> coa_c + elaidcrn_c 9315
HEXCCPT1: crn_c + hexccoa_c --> coa_c + hexccrn_c 9315
HPDCACRNCPT1: crn_c + hpdcacoa_c --> coa_c + hpdcacrn_c 9315
LNLNCGCPT1: crn_c + lnlncgcoa_c --> coa_c + lnlncgcrn_c 9315
NRVNCCPT1: crn_c + nrvnccoa_c --> coa_c + nrvnccrn_c 9315
PTDCACRNCPT1: crn_c + ptdcacoa_c --> coa_c + ptdcacrn_c 9315
RTOTAL2CRNCPT1: Rtotal2coa_c + crn_c --> Rtotal2crn_c + coa_c 9315
RTOTAL3CRNCPT1: Rtotal3coa_c + crn_c --> Rtotal3cr

In [21]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('choline O-acetyltransferase')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
9315,K00623: CHAT; choline O-acetyltransferase,S,,"CPT1A,CPT1B,CPT1C",CAT2,CRAT,Not Essential,"mito 24.5, cyto_mito 13.5",AKA*
15783,K00623: CHAT; choline O-acetyltransferase,,,,CAT2,CRAT,Not Essential,mito 25,AKL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
9315,0.100782,0.0170754,0.0600288,0.236768,0.0167474,0.169648,0.0449251,-0.315115,0.690845,0.75223,0.0392563


In [22]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ARACHCPT1 arachcoa_c + crn_c --> arachcrn_c + coa_c 9315
C160CPT1 crn_c + pmtcoa_c --> coa_c + pmtcrn_c 9315
C161CPT1 crn_c + hdcoa_c --> coa_c + hdcecrn_c 9315
C161CPT12 crn_c + hdd2coa_c --> coa_c + hdd2crn_c 9315
C181CPT1 crn_c + odecoa_c --> coa_c + odecrn_c 9315
CLPNDCPT1 clpndcoa_c + crn_c --> clpndcrn_c + coa_c 9315
DCSPTN1CPT1 crn_c + dcsptn1coa_c --> coa_c + dcsptn1crn_c 9315
ELAIDCPT1 crn_c + od2coa_c --> coa_c + elaidcrn_c 9315
HEXCCPT1 crn_c + hexccoa_c --> coa_c + hexccrn_c 9315
HPDCACRNCPT1 crn_c + hpdcacoa_c --> coa_c + hpdcacrn_c 9315
LNLNCGCPT1 crn_c + lnlncgcoa_c --> coa_c + lnlncgcrn_c 9315
NRVNCCPT1 crn_c + nrvnccoa_c --> coa_c + nrvnccrn_c 9315
PTDCACRNCPT1 crn_c + ptdcacoa_c --> coa_c + ptdcacrn_c 9315
RTOTAL2CRNCPT1 Rtotal2coa_c + crn_c --> Rtotal2crn_c + coa_c 9315
RTOTAL3CRNCPT1 Rtotal3coa_c + crn_c --> Rtotal3crn_c + coa_c 9315
RTOTALCRNCPT1 Rtotalcoa_c + crn_c --> Rtotalcrn_c + coa_c 9315
TETTET6CPT1 crn_c + tettet6coa_c --> coa_c + tettet6crn_c 9315
TTDCPT1 

In [23]:
model.remove_reactions(['RTOTAL2CRNCPT1','RTOTAL3CRNCPT1','RTOTALCRNCPT1',
                        'RTOTAL2CRNCPT2','RTOTAL3CRNCPT2','RTOTALCRNCPT2',
                        'RTOTAL2CRNt','RTOTAL3CRNt','RTOTALCRNt'],remove_orphans=True)

In [24]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('carnitine')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
9331,"K15109: SLC25A20_29, CACT, CACL, CRC1; solute ...",,CRC1,SLC25A20,CRC1,SLC25A,Not Essential,"mito 13.5, cyto_mito 12.5, cyto 10.5",ALF*
10149,"K15109: SLC25A20_29, CACT, CACL, CRC1; solute ...",,,,YMC2,SLC25,Not Essential,"mito 11, plas 10, nucl 1, cyto 1, extr 1, cyto...",TSD*
12591,KOG3957: Predicted L-carnitine dehydratase/alp...,S,,,,SUGCT,Not Essential,"mito 11, extr 5, cyto 4.5, cyto_nucl 4.5, nucl...",TSE*
13580,K00624: E2.3.1.7; carnitine O-acetyltransferase,,YAT1,CPT2,YAT1,CPT2,Not Essential,"cyto 12, cysk 6, nucl 5, mito 3",DFV*
13666,KOG0758: Mitochondrial carnitine-acylcarnitine...,,,,,SLC25,Not Essential,"mito 18, cyto 4, extr 4",GVG*
14245,K00624: E2.3.1.7; carnitine O-acetyltransferase,,CAT2,CRAT,CAT2,CRAT,Not Essential,"nucl 11.5, cyto_nucl 8, pero 6, mito 5, cyto 3.5",AKL*
15390,KOG0758: Mitochondrial carnitine-acylcarnitine...,,,,ORT1,SLC25,Not Essential,"cyto_mito 10.332, mito 10, cyto 10, cyto_nucl 9.5",RIE*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
9331,0.317304,0.660965,-0.231468,-1.77645,0.262022,-1.83089,-3.79725,-6.14964,-0.943653,-0.572548,-1.19392
10149,-0.075474,-0.4456,-0.241651,0.0656419,-0.293397,-0.0232707,-0.142486,0.752866,-0.199261,-0.496042,-0.194102
12591,0.194622,0.213333,0.153395,0.184751,0.0930738,0.0114473,0.11552,-0.453092,-0.126999,-0.166318,0.367602
13580,-0.239219,3.78134e-05,-0.00584095,-1.86406,-0.310162,-0.0740489,-0.0123864,0.374486,0.231935,0.164481,0.439824
14245,0.0908612,0.110605,0.0366664,-0.673789,-1.23329,-1.08932,-2.16438,-2.74615,-0.209163,-0.174774,0.0767215
15390,0.0647533,0.144211,-0.0323675,-0.139986,-0.405733,0.0388373,-0.199415,-0.347569,-0.159976,0.0398413,0.038154


In [25]:
print(Annotation.loc['9331','Combined Annotations'])
print(Annotation.loc['10149','WolfPSort'])
print(Annotation.loc['12591','Combined Annotations'])
print(Annotation.loc['12591','WolfPSort'])
print(Annotation.loc['13666','Combined Annotations'])

K15109: SLC25A20_29, CACT, CACL, CRC1; solute carrier family 25 (mitochondrial carnitine/acylcarnitine transporter), member 20/29
mito 11, plas 10, nucl 1, cyto 1, extr 1, cyto_nucl 1, pero 1, E.R. 1, golg 1, cyto_pero 1
KOG3957: Predicted L-carnitine dehydratase/alpha-methylacyl-CoA racemase
mito 11, extr 5, cyto 4.5, cyto_nucl 4.5, nucl 3.5
KOG0758: Mitochondrial carnitine-acylcarnitine carrier protein


In [26]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ACRNtm acrn_c --> acrn_m 9331
ARACHCRNt arachcrn_c --> arachcrn_m 9331
C160CRNt pmtcrn_c --> pmtcrn_m 9331
C161CRN2t hdd2crn_c --> hdd2crn_m 9331
C161CRNt hdcecrn_c --> hdcecrn_m 9331
C181CRNt odecrn_c --> odecrn_m 9331
CLPNDCRNt clpndcrn_c --> clpndcrn_m 9331
CRNCARtm acrn_c + crn_m --> acrn_m + crn_c 9331
CRNtim crn_m --> crn_c 9331
DCSPTN1CRNt dcsptn1crn_c --> dcsptn1crn_m 9331
ELAIDCRNt elaidcrn_c --> elaidcrn_m 9331
HEXCCRNt hexccrn_c --> hexccrn_m 9331
HPDCACRNt hpdcacrn_c --> hpdcacrn_m 9331
LNLNCGCRNt lnlncgcrn_c --> lnlncgcrn_m 9331
NRVNCCRNt nrvnccrn_c --> nrvnccrn_m 9331
PTDCACRNt ptdcacrn_c --> ptdcacrn_m 9331
TETTET6CRNt tettet6crn_c --> tettet6crn_m 9331
TTDCRNt ttdcrn_c --> ttdcrn_m 9331

10149 no reactions

12591 no reactions

ARACHCPT2 arachcrn_m + coa_m --> arachcoa_m + crn_m 13580
C160CPT2 coa_m + pmtcrn_m --> crn_m + pmtcoa_m 13580
C161CPT2 coa_m + hdcecrn_m --> crn_m + hdcoa_m 13580
C161CPT22 coa_m + hdd2crn_m --> crn_m + hdd2coa_m 13580
C181CPT2 coa_m + odecrn_m -

In [27]:
# CRNCARtm redundant with ACRNtm and CRNtim
# CSNATirm redundant with CSNATm
# CRAT is not in cytosol remove CSNATr
model.remove_reactions(['CRNCARtm','CSNATirm','CSNATr'],remove_orphans=True)

cyto carnitine acyltransferase CPT1 9315 (anchored in mito outermembrane facing cytosol)  
mito carnitine acyltransferase CPT2 13580  
pero carnitine acyltransferase 15783

mitochondrial/peroxisome/ER carnitine acetyltransferase CRAT 14245

mito carnitine transport 9331
pero carnitine transport 10149

In [28]:
for m in model.metabolites:
    if m.id.endswith('crn_x'):
        for r in m.reactions:
            print(r, r.gene_reaction_rule)

CSNATp: accoa_x + crn_x <=> acrn_x + coa_x 14245
CSNATp: accoa_x + crn_x <=> acrn_x + coa_x 14245
CSNAT2x: crn_x + dmnoncoa_x <=> coa_x + dmnoncrn_x 14245
CSNAT3x: crn_x + ppcoa_x <=> coa_x + pcrn_x 14245
CSNAT2x: crn_x + dmnoncoa_x <=> coa_x + dmnoncrn_x 14245
CSNAT3x: crn_x + ppcoa_x <=> coa_x + pcrn_x 14245


In [29]:
model.reactions.get_by_id('CSNAT2x').gene_reaction_rule = '15783'
# pcrn cyto transferase is missing
# transport of acrn and pcrn from pero and into mito is missing
r1 = hsa2.reactions.get_by_id('C30CPT1').copy()
r1.gene_reaction_rule = '9315'
r2 = hsa2.reactions.get_by_id('PCRNtm').copy()
r2.gene_reaction_rule = '9331'
r3 = hsa2.reactions.get_by_id('PCRNtc').copy()
r3.gene_reaction_rule = '10149'
r4 = sce.reactions.get_by_id('ACRNtp').copy()
r4.gene_reaction_rule = '10149'
model.add_reactions([r1,r2,r3,r4])
# also add medium chain crn transport from pero to cyto? further degradation in mito

ER and pero fatty acid transport  
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2846691/  
ER  
10946 ortholog of SLC33A1 ER acyl-coa transporter  
9912 FATP4 http://jcs.biologists.org/content/119/22/4678  
AtABCA9 https://www.pnas.org/content/110/2/773  
AT-1 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2939804/  
pero  
9637 PXA1 and 13167 PXA2 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3370197/  

In [30]:
temp = ['10946','9912']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10946,KOG3574: Acetyl-CoA transporter,,YBR220C,SLC33A1,YBR220C,SLC33,Not Essential,plas 26,NGR*
9912,"K08745: SLC27A1_4, FATP1, FATP4; solute carrie...",S,FAT1,"SLC27A1,SLC27A4,SLC27A2,SLC27A5,SLC27A6",FAT1,SLC27,Not Essential,"extr 12, plas 8, pero 4, mito 1, cyto 1, E.R. ...",VRL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
10946,0.0358943,0.227035,0.149761,0.0525809,-0.825083,0.314581,0.515223,0.992047,0.412156,-0.0177003,0.296114
9912,0.0677425,-0.270103,-0.302936,-0.051684,-0.404229,0.0682701,-0.0630695,-0.272301,-0.340334,-0.0743227,-0.0288945


In [31]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ACCOAgt accoa_c <=> accoa_g 10946
ACCOAtm accoa_c + coa_m <=> accoa_m + coa_c 10946
ACCOAtr accoa_c <=> accoa_r 10946
ACCOAtx accoa_c + coa_x <=> accoa_x + coa_c 10946

FA240tp ttc_c --> ttc_x 9912
FA260tp hexc_c --> hexc_x 9912
FACOAL240p atp_x + coa_x + ttc_x --> amp_x + ppi_x + ttccoa_x 9912
FACOAL260p atp_x + coa_x + hexc_x --> amp_x + hexccoa_x + ppi_x 9912
FATP1t hdca_c + na1_c <=> hdca_e + na1_e 9912
FATP2t na1_c + ocdcea_c <=> na1_e + ocdcea_e 9912
FATP3t na1_c + ocdca_c <=> na1_e + ocdca_e 9912
FATP4t arach_c + na1_c <=> arach_e + na1_e 9912
FATP8t na1_c + ttc_c <=> na1_e + ttc_e 9912
FATP9t na1_c + nrvnc_c <=> na1_e + nrvnc_e 9912
VLCS2p atp_x + coa_x + dhcholestanate_x --> amp_x + dhcholestancoa_x + ppi_x 9912
VLCS2r atp_r + coa_r + dhcholestanate_r --> amp_r + dhcholestancoa_r + ppi_r 9912
VLCSp atp_x + coa_x + thcholstoic_x --> amp_x + cholcoar_x + ppi_x 9912
VLCSr atp_r + coa_r + thcholstoic_r --> amp_r + cholcoar_r + ppi_r 9912



In [32]:
model.remove_reactions(['ACCOAtm','ACCOAtx'],remove_orphans=True)

In [33]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if m.id.endswith('coa_r') and not m.id == 'coa_r' and not m.id == 'accoa_r':
        print(m)
        for r in m.reactions:
            print(r, r.gene_reaction_rule)
        print()

dcacoa_r
TRIGSer_RT: 0.01 12dgr_RT_r + 0.02 dcacoa_r + 0.06 ddcacoa_r + 0.17 hdcoa_r + 0.09 ocdycacoa_r + 0.24 odecoa_r + 0.27 pmtcoa_r + 0.05 stcoa_r + 0.1 tdcoa_r --> coa_r + 0.01 triglyc_RT_r 16460
AGATer_RT: 0.01 1ag3p_RT_r + 0.02 dcacoa_r + 0.06 ddcacoa_r + 0.17 hdcoa_r + 0.09 ocdycacoa_r + 0.24 odecoa_r + 0.27 pmtcoa_r + 0.05 stcoa_r + 0.1 tdcoa_r --> coa_r + 0.01 pa_RT_r 10427 or 16030 or 16779 or 9746
LPCATer_RT: 0.01 1agpc_RT_r + 0.02 dcacoa_r + 0.06 ddcacoa_r + 0.17 hdcoa_r + 0.09 ocdycacoa_r + 0.24 odecoa_r + 0.27 pmtcoa_r + 0.05 stcoa_r + 0.1 tdcoa_r --> coa_r + 0.01 pc_RT_r 16030
GAT2er_RT: 0.02 dcacoa_r + 0.06 ddcacoa_r + dhap_r + 0.17 hdcoa_r + 0.09 ocdycacoa_r + 0.24 odecoa_r + 0.27 pmtcoa_r + 0.05 stcoa_r + 0.1 tdcoa_r --> 0.01 1agly3p_RT_r + coa_r 13369 or 15435
GAT1er_RT: 0.02 dcacoa_r + 0.06 ddcacoa_r + glyc3p_r + 0.17 hdcoa_r + 0.09 ocdycacoa_r + 0.24 odecoa_r + 0.27 pmtcoa_r + 0.05 stcoa_r + 0.1 tdcoa_r --> 0.01 1ag3p_RT_r + coa_r 15435

ddcacoa_r
TRIGSer_RT: 0.01

In [34]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if m.id.endswith('coa_r') and not m.id == 'coa_r' and not m.id == 'accoa_r' and not m.id.startswith('dhchol'):
        print(m, m.name, m.id.upper().replace('_R','tr'), m.id.upper().replace('_R','tr') in model.reactions)

dcacoa_r Decanoyl-CoA (n-C10:0CoA) DCACOAtr False
ddcacoa_r Lauroyl-CoA DDCACOAtr False
hdcoa_r Hexadecenoyl-CoA (n-C16:1CoA) HDCOAtr False
hexccoa_r Hexacosanoyl CoA  n C260CoA  C47H82N7O17P3S HEXCCOAtr False
hmgcoa_r Hydroxymethylglutaryl CoA C27H39N7O20P3S HMGCOAtr False
lnlncgcoa_r Gamma-linolenoyl-CoA LNLNCGCOAtr False
malcoa_r Malonyl CoA C24H33N7O19P3S MALCOAtr False
ocdycacoa_r Octadecynoyl CoA  n C182CoA  C39H62N7O17P3S OCDYCACOAtr False
odecoa_r Octadecenoyl-CoA (n-C18:1CoA) ODECOAtr False
pmtcoa_r Palmitoyl-CoA PMTCOAtr False
stcoa_r Stearoyl-CoA STCOAtr False
tdcoa_r Tetradecanoyl-CoA TDCOAtr False
ttccoa_r Tetracosanoyl CoA   n C240CoA  C45H78N7O17P3S TTCCOAtr False


In [35]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if m.id.endswith('coa_r') and not m.id == 'coa_r' and not m.id == 'accoa_r' and not m.id.startswith('dhchol'):
        print('model.reactions.get_by_id(\''+m.id.upper().replace('_R','tr')+'\').name = \''+m.name+' transport, endoplasmic reticulum\'')

model.reactions.get_by_id('DCACOAtr').name = 'Decanoyl-CoA (n-C10:0CoA) transport, endoplasmic reticulum'
model.reactions.get_by_id('DDCACOAtr').name = 'Lauroyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('HDCOAtr').name = 'Hexadecenoyl-CoA (n-C16:1CoA) transport, endoplasmic reticulum'
model.reactions.get_by_id('HEXCCOAtr').name = 'Hexacosanoyl CoA  n C260CoA  C47H82N7O17P3S transport, endoplasmic reticulum'
model.reactions.get_by_id('HMGCOAtr').name = 'Hydroxymethylglutaryl CoA C27H39N7O20P3S transport, endoplasmic reticulum'
model.reactions.get_by_id('LNLNCGCOAtr').name = 'Gamma-linolenoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('MALCOAtr').name = 'Malonyl CoA C24H33N7O19P3S transport, endoplasmic reticulum'
model.reactions.get_by_id('OCDYCACOAtr').name = 'Octadecynoyl CoA  n C182CoA  C39H62N7O17P3S transport, endoplasmic reticulum'
model.reactions.get_by_id('ODECOAtr').name = 'Octadecenoyl-CoA (n-C18:1CoA) transport, endoplasmic reticulum

In [36]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if m.id.endswith('coa_r') and not m.id == 'coa_r' and not m.id == 'accoa_r' and not m.id.startswith('dhchol'):
        r = model.reactions.get_by_id('ACCOAtr').copy()
        r.id = m.id.upper().replace('_R','tr')
        model.add_reactions([r])
        r.add_metabolites({'accoa_c': 1.0, 'accoa_r': -1.0, m.id: 1.0, m.id.replace('_r','_c'): -1.0})

model.reactions.get_by_id('DCACOAtr').name = 'Decanoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('DDCACOAtr').name = 'Lauroyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('HDCOAtr').name = 'Hexadecenoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('HEXCCOAtr').name = 'Hexacosanoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('HMGCOAtr').name = 'Hydroxymethylglutaryl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('LNLNCGCOAtr').name = 'Gamma-linolenoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('MALCOAtr').name = 'Malonyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('OCDYCACOAtr').name = 'Octadecynoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('ODECOAtr').name = 'Octadecenoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('PMTCOAtr').name = 'Palmitoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('STCOAtr').name = 'Stearoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('TDCOAtr').name = 'Tetradecanoyl-CoA transport, endoplasmic reticulum'
model.reactions.get_by_id('TTCCOAtr').name = 'Tetracosanoyl-CoA transport, endoplasmic reticulum'

In [37]:
for r in sorted(model.metabolites.pmtcoa_x.reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

ACACT8p: 3ohodcoa_x + coa_x --> accoa_x + pmtcoa_x 13813
ACOAD7p: fad_x + pmtcoa_x --> fadh2_x + hdd2coa_x 12989 or 16253
ACOAO7p: o2_x + pmtcoa_x --> h2o2_x + hdd2coa_x 12742 or 12752 or 9700
FA160COAabcp: atp_x + h2o_x + pmtcoa_c --> adp_x + h_x + pi_x + pmtcoa_x 13167 or 9637
FA160COAabcp_1: atp_c + h2o_c + pmtcoa_c --> adp_c + h_c + pi_c + pmtcoa_x 13167 or 9637
FACOAL160p: atp_x + coa_x + hdca_x --> amp_x + pmtcoa_x + ppi_x 12555 or 15900
PTE2x: h2o_x + pmtcoa_x --> coa_x + h_x + hdca_x 16048


In [38]:
temp = ['9637','13167','12538','12555','15900','9912']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
9637,"K15628: PXA; ATP-binding cassette, subfamily D...",S,"PXA1,PXA2","ABCD1,ABCD2,ABCD3",PXA1,ABCD2,Not Essential,"extr 14, mito 6, plas 4, cyto_mito 4",KET*
13167,"K05676: ABCD2, ALDL1; ATP-binding cassette, su...",,"PXA1,PXA2","ABCD1,ABCD2,ABCD3",PXA2,ABCD2,Not Essential,"plas 10, mito 8, cyto 4, pero 2, nucl 1, extr ...",ERI*
12538,KOG1256: Long-chain acyl-CoA synthetases (AMP-...,,FAA2,"ACSL1,ACSL5,ACSL6,CTB-127M13.1",FAA2,ACSL5,Not Essential,"mito 18.5, cyto_mito 12.5, cyto 5.5",GEQ*
12555,"K01897: ACSL, fadD; long-chain acyl-CoA synthe...",,FAA2,"ACSL1,ACSL5,ACSL6,CTB-127M13.1",FAA2,ACSL1,Not Essential,"cyto 15.5, cyto_nucl 11.5, pero 6, nucl 4.5",AKL*
15900,K01896: ACSM; medium-chain acyl-CoA synthetase,,,"ACSM1,ACSM2A,ACSM2B,ACSM3,ACSM4,ACSM5",ACS2,ACSM3,Not Essential,"cyto 13, pero 11, cyto_nucl 8.5",AKL*
9912,"K08745: SLC27A1_4, FATP1, FATP4; solute carrie...",S,FAT1,"SLC27A1,SLC27A4,SLC27A2,SLC27A5,SLC27A6",FAT1,SLC27,Not Essential,"extr 12, plas 8, pero 4, mito 1, cyto 1, E.R. ...",VRL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
9637,0.0387351,0.118884,-0.169197,-0.160041,-0.191542,-0.00882013,-0.986535,-0.0397322,-0.0791493,0.151542,-0.0321814
13167,-0.000429381,0.137701,-0.0476588,0.249415,0.117662,0.180606,-0.731184,-0.0962394,-0.0646791,-0.146305,-0.0691314
12538,-0.0228742,-0.00284459,0.0444012,0.0245939,0.0476792,-0.0249695,0.0981869,0.240673,0.0573442,0.0435764,0.0668027
12555,-0.311448,-0.162485,-0.131826,-0.274268,-0.854248,-0.027038,-1.52861,-0.659443,-0.300839,-0.371389,-0.434224
15900,-0.402251,0.0381164,-0.120976,-0.323788,-0.290967,-0.292245,0.0775032,0.423076,0.367482,-0.304034,-0.394707
9912,0.0677425,-0.270103,-0.302936,-0.051684,-0.404229,0.0682701,-0.0630695,-0.272301,-0.340334,-0.0743227,-0.0288945


In [39]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

FA140COAabcp atp_x + h2o_x + tdcoa_c --> adp_x + h_x + pi_x + tdcoa_x 13167 or 9637
FA141COAabcp atp_x + h2o_x + tdecoa_c --> adp_x + h_x + pi_x + tdecoa_x 13167 or 9637
FA160COAabcp atp_x + h2o_x + pmtcoa_c --> adp_x + h_x + pi_x + pmtcoa_x 13167 or 9637
FA160COAabcp_1 atp_c + h2o_c + pmtcoa_c --> adp_c + h_c + pi_c + pmtcoa_x 13167 or 9637
FA161COAabcp atp_x + h2o_x + hdcoa_c --> adp_x + h_x + hdcoa_x + pi_x 13167 or 9637
FA180COAabcp atp_x + h2o_x + stcoa_c --> adp_x + h_x + pi_x + stcoa_x 13167 or 9637
FA181COAabcp atp_x + h2o_x + odecoa_c --> adp_x + h_x + odecoa_x + pi_x 13167 or 9637
FA182COAabcp atp_x + h2o_x + ocdycacoa_c --> adp_x + h_x + ocdycacoa_x + pi_x 13167 or 9637

FA140COAabcp atp_x + h2o_x + tdcoa_c --> adp_x + h_x + pi_x + tdcoa_x 13167 or 9637
FA141COAabcp atp_x + h2o_x + tdecoa_c --> adp_x + h_x + pi_x + tdecoa_x 13167 or 9637
FA160COAabcp atp_x + h2o_x + pmtcoa_c --> adp_x + h_x + pi_x + pmtcoa_x 13167 or 9637
FA160COAabcp_1 atp_c + h2o_c + pmtcoa_c --> adp_c + h

In [40]:
# fatty acid peroxisomal transport
model.remove_reactions(['FA160COAabcp_1'],remove_orphans=True)
# acyl-coa is hydrolyzed and transported by pxa1 and pxa2
# active domain on cytosolic side
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FA') and r.id.endswith('abcp'):
        r.gene_reaction_rule = '13167 and 9637'
        r.add_metabolites({'atp_x': 1.0, 'h2o_x': 1.0, 'adp_x': -1.0, 'h_x': -1.0, 'pi_x': -1.0,
                           'atp_c': -1.0, 'h2o_c': -1.0, 'adp_c': 1.0, 'h_c': 1.0, 'pi_c': 1.0})
model.reactions.get_by_id('FA140COAabcp').add_metabolites({'tdcoa_x': -1.0, 'ttdca_x': 1.0,
                                                           'h2o_c': -1.0, 'coa_c': 1.0, 'h_c': 1.0})
model.reactions.get_by_id('FA141COAabcp').add_metabolites({'tdecoa_x': -1.0, 'ttdcea_x': 1.0,
                                                           'h2o_c': -1.0, 'coa_c': 1.0, 'h_c': 1.0})
model.reactions.get_by_id('FA160COAabcp').add_metabolites({'pmtcoa_x': -1.0, 'hdca_x': 1.0,
                                                           'h2o_c': -1.0, 'coa_c': 1.0, 'h_c': 1.0})
model.reactions.get_by_id('FA161COAabcp').add_metabolites({'hdcoa_x': -1.0, 'hdcea_x': 1.0,
                                                           'h2o_c': -1.0, 'coa_c': 1.0, 'h_c': 1.0})
model.reactions.get_by_id('FA180COAabcp').add_metabolites({'stcoa_x': -1.0, 'ocdca_x': 1.0,
                                                           'h2o_c': -1.0, 'coa_c': 1.0, 'h_c': 1.0})
m1 = model.metabolites.get_by_id('ocdcea_c').copy()
m1.id = 'ocdcea_x'
m1.compartment = 'x'
m2 = model.metabolites.get_by_id('ocdcya_c').copy()
m2.id = 'ocdcya_x'
m2.compartment = 'x'
model.add_metabolites([m1,m2])
model.reactions.get_by_id('FA181COAabcp').add_metabolites({'odecoa_x': -1.0, 'ocdcea_x': 1.0,
                                                           'h2o_c': -1.0, 'coa_c': 1.0, 'h_c': 1.0})
model.reactions.get_by_id('FA182COAabcp').add_metabolites({'ocdycacoa_x': -1.0, 'ocdcya_x': 1.0,
                                                           'h2o_c': -1.0, 'coa_c': 1.0, 'h_c': 1.0})
r = model.reactions.get_by_id('FA140COAabcp').copy()
r.id = 'FA183COAabcp'
model.add_reactions([r])
m = model.metabolites.get_by_id('lnlncg_c').copy()
m.id = 'lnlncg_x'
m.compartment = 'x'
model.add_metabolites([m])
r.add_metabolites({'tdcoa_c': 1.0, 'ttdca_x': -1.0, 'lnlncgcoa_c': -1.0, 'lnlncg_x': 1.0})

# free fatty acid is activated again by 12555 pero faa2
# 15900 ACSM is for C4 to C11
model.reactions.get_by_id('FACOAL80p').gene_reaction_rule = '15900'
model.reactions.get_by_id('FACOAL100p').gene_reaction_rule = '15900'
model.reactions.get_by_id('FACOAL120p').gene_reaction_rule = '12555'
model.reactions.get_by_id('FACOAL140p').gene_reaction_rule = '12555'
model.reactions.get_by_id('FACOAL141p').gene_reaction_rule = '12555'
model.reactions.get_by_id('FACOAL160p').gene_reaction_rule = '12555'
model.reactions.get_by_id('FACOAL161p').gene_reaction_rule = '12555'
for x in ['FACOAL180','FACOAL181','FACOAL182','FACOAL1831']:
    r = model.reactions.get_by_id(x).copy()
    r.id = r.id + 'p'
    r.gene_reaction_rule = '12555'
    model.add_reactions([r])
    for m in r.metabolites:
        if not m.id.replace('_c','_x') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_x')
            m2.compartment = 'x'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_x'): r.get_coefficient(m.id)})
# free very long chain fatty acid transported and activated by 9912 fat1
# very long chain fatty acid also activated by faa2
model.reactions.get_by_id('FACOAL240p').gene_reaction_rule = '12555 or 9912'
model.reactions.get_by_id('FACOAL260p').gene_reaction_rule = '12555 or 9912'

In [41]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

FA140COAabcp atp_c + 2.0 h2o_c + tdcoa_c --> adp_c + coa_c + 2.0 h_c + pi_c + ttdca_x 13167 and 9637
FA141COAabcp atp_c + 2.0 h2o_c + tdecoa_c --> adp_c + coa_c + 2.0 h_c + pi_c + ttdcea_x 13167 and 9637
FA160COAabcp atp_c + 2.0 h2o_c + pmtcoa_c --> adp_c + coa_c + 2.0 h_c + hdca_x + pi_c 13167 and 9637
FA161COAabcp atp_c + 2.0 h2o_c + hdcoa_c --> adp_c + coa_c + 2.0 h_c + hdcea_x + pi_c 13167 and 9637
FA180COAabcp atp_c + 2.0 h2o_c + stcoa_c --> adp_c + coa_c + 2.0 h_c + ocdca_x + pi_c 13167 and 9637
FA181COAabcp atp_c + 2.0 h2o_c + odecoa_c --> adp_c + coa_c + 2.0 h_c + ocdcea_x + pi_c 13167 and 9637
FA182COAabcp atp_c + 2.0 h2o_c + ocdycacoa_c --> adp_c + coa_c + 2.0 h_c + ocdcya_x + pi_c 13167 and 9637
FA183COAabcp atp_c + 2.0 h2o_c + lnlncgcoa_c --> adp_c + coa_c + 2.0 h_c + lnlncg_x + pi_c 13167 and 9637

FA140COAabcp atp_c + 2.0 h2o_c + tdcoa_c --> adp_c + coa_c + 2.0 h_c + pi_c + ttdca_x 13167 and 9637
FA141COAabcp atp_c + 2.0 h2o_c + tdecoa_c --> adp_c + coa_c + 2.0 h_c + pi_c

In [42]:
r = model.reactions.get_by_id('BIOMASS_RT')
for m in sorted(r.metabolites, key=lambda x: x.id):
    print(m, r.get_coefficient(m.id))

13BDglcn_c -1.1348
adp_c 59.276
ala__L_c -0.4588
amp_c -0.046
arg__L_c -0.1607
asn__L_c -0.1017
asp__L_c -0.2975
atp_c -59.276
cmp_c -0.0447
cys__L_c -0.0066
damp_c -0.0036
dcmp_c -0.0024
dgmp_c -0.0024
dtmp_c -0.0036
ergst_c -0.0007
gln__L_c -0.1054
glu__L_c -0.3018
gly_c -0.2904
glycogen_c -0.5185
gmp_c -0.046
h2o_c -59.276
h_c 58.70001
his__L_c -0.0663
ile__L_c -0.1927
leu__L_c -0.2964
lys__L_c -0.2862
mannan_r -0.8079
met__L_c -0.0507
pa_RT_r -6e-06
pc_RT_r -6e-05
pe_RT_r -4.5e-05
phe__L_c -0.1339
pi_c 59.305
pro__L_c -0.1647
ps_RT_r -1.7e-05
ptd1ino_RT_r -5.3e-05
ribflv_c -0.00099
ser__L_c -0.1854
so4_c -0.02
thr__L_c -0.1914
tre_c -0.0234
triglyc_RT_r -6.6e-05
trp__L_c -0.0284
tyr__L_c -0.102
ump_c -0.0599
val__L_c -0.2646
zymst_c -0.0015


In [43]:
r = model.reactions.get_by_id('BIOMASS_RT')
# Change sterols in biomass to ER
for x in ['zymst_c','ergst_c']:
    r.add_metabolites({x: -r.get_coefficient(x),
                       x.replace('_c','_r'): r.get_coefficient(x)})

In [44]:
r = model.reactions.get_by_id('BIOMASS_RT')
for m in r.metabolites:
    if len(m.reactions) < 2:
        print(m.id)

In [45]:
temp = ['16430','10400','12097','10888','14882','16411','13481','15169']
display(Annotation.loc[temp])
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
16430,KOG1441: Glucose-6-phosphate/phosphate and pho...,,,,,SLC35,Not Essential,plas 26,RLG*
10400,KOG1441: Glucose-6-phosphate/phosphate and pho...,S,,,,SLC35,Not Essential,"plas 19, E.R. 6",KQQ*
12097,"K15280: SLC35C2; solute carrier family 35, mem...",,,,,SLC35,Not Essential,"plas 21, mito 3, E.R. 2",KLG*
10888,KOG1441: Glucose-6-phosphate/phosphate and pho...,A,,,,SLC35,Not Essential,plas 25,VGR*
14882,KOG1441: Glucose-6-phosphate/phosphate and pho...,A,,,,SLC35,Not Essential,"plas 17, E.R. 5, mito 4",TPN*
16411,"K15283: SLC35E1; solute carrier family 35, mem...",,SLY41,SLC35E1,SLY41,SLC35,Not Essential,"plas 21, nucl 3, mito 1, extr 1, vacu 1",QLR*
13481,KOG1441: Glucose-6-phosphate/phosphate and pho...,,,,,,Not Essential,"plas 12, extr 5, mito 4, E.R. 4",FLD*
15169,"K15279: SLC35C1, FUCT1; solute carrier family ...",,,SLC35C1,,SLC35C,Not Essential,plas 26,ERR*


16430 no reactions

10400 no reactions

12097 no reactions

10888 no reactions

14882 no reactions

16411 no reactions

13481 no reactions

GDPFUCtg: gdpfuc_c + gmp_g <=> gdpfuc_g + gmp_c 15169



In [46]:
r = cobra.Reaction('GLYC3Pter')
r.name = 'Glycerol 3-phosphate endoplasmic reticular transport'
#r.gene_reaction_rule = ''
r.lower_bound = -1000.0
r.upper_bound = 1000.0
r.add_metabolites({model.metabolites.glyc3p_c: -1.0, model.metabolites.glyc3p_r: 1.0})
model.add_reactions([r])

r = cobra.Reaction('DHAPter')
r.name = 'Dihydroxyacetone phosphate endoplasmic reticular transport'
#r.gene_reaction_rule = ''
r.lower_bound = -1000.0
r.upper_bound = 1000.0
r.add_metabolites({model.metabolites.dhap_c: -1.0, model.metabolites.dhap_r: 1.0})
model.add_reactions([r])

In [47]:
model.objective = 'BIOMASS_RT'

In [48]:
for x in sorted(model.reactions, key=lambda x: x.id):
    if x.boundary:
        print(x.id, x.lower_bound, x.upper_bound)

EX_co2_e 0.0 999999.0
EX_glc__D_e -10.0 999999.0
EX_h2o_e -999999.0 999999.0
EX_h_e -999999.0 999999.0
EX_nh4_e -999999.0 999999.0
EX_o2_e -2.0 999999.0
EX_pi_e -999999.0 999999.0
EX_so4_e -999999.0 999999.0


In [49]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if 'e' in m.compartment:
        if 'EX_'+m.id in model.reactions:
            print(m.id)

co2_e
glc__D_e
h2o_e
h_e
nh4_e
o2_e
pi_e
so4_e


In [50]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if 'e' in m.compartment:
        if not 'EX_'+m.id in model.reactions:
            print(m.id)

13BDglcn_e
35cgmp_e
4abut_e
5adtststeroneglc_e
5aop_e
5flura_e
6mpur_e
ac_e
acgam_e
ach_e
ade_e
adn_e
ahandrostanglc_e
ala_B_e
ala__D_e
ala__L_e
alltn_e
alltt_e
amet_e
andrstrnglc_e
arach_e
arg__L_e
asn__L_e
asp__L_e
bildglcur_e
bilglcur_e
ca2_e
camp_e
cgly_e
chol_e
cholate_e
chtn_e
cl_e
csn_e
cu_e
cys__L_e
cysi__L_e
cytd_e
dad_2_e
dca_e
ddca_e
dhdascb_e
din_e
epist_e
ergst_e
estradiolglc_e
estriolglc_e
estroneglc_e
fe2_e
fe3_e
fecost_e
fmn_e
for_e
fru_e
g3pc_e
g3pe_e
g3pi_e
gal_e
galur_e
gam_e
gchola_e
gln__L_e
glu__L_e
gluala_e
gly_e
glyc_e
glygn2_e
glygn4_e
gthrd_e
gua_e
hco3_e
hdca_e
hdcea_e
his__L_e
ile__L_e
inost_e
ins_e
k_e
lac__D_e
lac__L_e
lanost_e
leu__L_e
lys__L_e
malt_e
man_e
met__L_e
na1_e
nac_e
no2_e
no3_e
nrvnc_e
ocdca_e
ocdcea_e
ocdcya_e
oh1_e
orn_e
oxa_e
pail1819Z160_e
pc_RT_e
pe1801819Z_e
pe1801829Z12Z_e
pe1819Z1819Z_e
pe1819Z1829Z12Z_e
pe_RT_e
pectin_e
pepd_e
pg1819Z160_e
pgp1819Z160_e
phe__L_e
pnto__R_e
pro__D_e
pro__L_e
prostge1_e
prostge2_e
ptd1ino_RT_e
ptrc_e
pyr

In [51]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if 'e' in m.compartment:
        if not m.id.endswith('_e'):
            print(m.id)

In [52]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if 'e' in m.compartment:
        if 'EX_'+m.id not in model.reactions:
            r = model.reactions.get_by_id('EX_h_e').copy()
            r.id = 'EX_'+m.id
            r.name = m.name + ' exchange'
            model.add_reactions([r])
            model.reactions.get_by_id('EX_'+m.id).add_metabolites({'h_e': 1.0, m.id: -1.0})
            model.reactions.get_by_id('EX_'+m.id).lower_bound = 0.0

In [53]:
for r in model.reactions:
    if r.lower_bound < -1000.0:
        r.lower_bound = -1000.0
    if r.upper_bound > 1000.0:
        r.upper_bound = 1000.0

In [54]:
model.medium

{'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_glc__D_e': 10.0}

In [55]:
sol = model.optimize()
print(sol.objective_value)
for k, v in sol.shadow_prices.items():
    if v:
        print(k, v)

0.0
dolp_r 1.2377769525931428
mannan_r -1.2377769525931428


In [56]:
for r in sorted(model.metabolites.get_by_id('dolmanp_r').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dolp_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

DOLPMMer: dolmanp_r --> dolp_r + h_r + mannan_r 12121 or 13840 or 8768
DOLPMTcer: dolp_c + gdpmann_c --> dolmanp_r + gdp_c 13484

DOLPMT: dolp_c + gdpmann_c --> dolmanp_c + gdp_c 13484
DOLPMTcer: dolp_c + gdpmann_c --> dolmanp_r + gdp_c 13484
G12MT3: doldpglcnacglcnacman_manman_manmanman_c + dolmanp_c --> doldpglcnacglcnacman_manmanman_manmanman_c + dolp_c + h_c 9905
G12MT4: doldpglcnacglcnacman_man_man_manman_manmanman_c + dolmanp_c --> doldpglcnacglcnacman_man_manman_manman_manmanman_c + dolp_c + h_c 9905
GLCNACPT: dolp_c + 2.0 h_c + uacgam_c --> doldpglcnac_c + ump_c 11202
UDPDOLPT: dolp_c + udpg_c --> dolpglc_c + udp_c 11809


In [57]:
r = cobra.Reaction('DOLPter')
r.name = 'Dolichol phosphate flippase'
r.lower_bound = 0.0
r.upper_bound = 1000.0
r.add_metabolites({model.metabolites.dolp_r: -1.0, model.metabolites.dolp_c: 1.0})
model.add_reactions([r])

In [58]:
sol = model.optimize()
print(sol.objective_value)

for k, v in sol.shadow_prices.items():
    if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
        print(k, v)

0.0
ile__L_c -5.189413596263622


### amino acid transporters

In [59]:
temp = list(Annotation.index[Annotation['Combined Annotations'].str.contains('amino acid transporter')]) + \
       list(Annotation.index[Annotation['Combined Annotations'].str.contains('Amino acid transporter')]) + \
       ['15991','16258','9135','11017','13128','16454','11649','14774']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8962,K16261: YAT; yeast amino acid transporter,,DIP5,,DIP5,,Not Essential,plas 27,AIM*
9319,K16261: YAT; yeast amino acid transporter,A,DIP5,,DIP5,,Not Essential,"plas 21, mito 2, vacu 2",WLM*
9322,K16261: YAT; yeast amino acid transporter,A,DIP5,,DIP5,,Not Essential,plas 27,WLM*
9962,K16261: YAT; yeast amino acid transporter,S,DIP5,,DIP5,,Not Essential,"plas 21, E.R. 2, vacu 2",DAA*
10059,K16261: YAT; yeast amino acid transporter,,,,DIP5,,Not Essential,plas 27,YVL*
10669,HMMPfam:Transmembrane amino acid transporter p...,,,,,,Not Essential,plas 27,CAA*
11026,BLAST: transmembrane amino acid transporter pr...,,,,,,Not Essential,"plas 24, E.R. 2",VRG*
11625,K13869: SLC7A11; solute carrier family 7 (L-ty...,,,"SLC7A6,SLC7A7,SLC7A9,SLC7A5",MUP1,SLC7A,Not Essential,plas 27,EMT*
12743,"K14209: SLC36A, PAT; solute carrier family 36 ...",,"AVT3,AVT4","SLC36A1,SLC36A2,SLC36A3,SLC36A4",AVT3,SLC36,Not Essential,"plas 13, E.R. 6, extr 5, vacu 2",PRA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
8962,0.141486,0.0706017,-0.390702,-0.181357,-0.0403765,-0.347303,0.133173,0.00723612,-0.0278378,-0.0333285,0.404273
9319,0.28395,-0.120032,-0.0979726,0.212506,-0.238418,0.249639,-0.131144,-0.0190663,-0.498955,-0.281257,0.264164
9322,0.284803,0.183181,0.0765583,-0.0782087,-0.472852,-0.120008,-0.373087,-0.742947,-0.260077,-0.47357,-0.526786
9962,0.00129231,-0.155615,-0.265223,0.0187776,-0.279612,-0.163992,-0.205995,-0.156544,0.138236,0.0485481,0.402889
10059,0.147659,-0.475336,-0.336757,-0.0222465,0.0354311,0.201796,-0.224187,-0.162138,-0.0949714,-0.0628296,0.0267071
10669,-0.481501,-1.21922,-0.816654,-0.478422,-1.28689,-0.425725,0.546023,0.400691,0.417396,0.180457,0.146982
11026,0.0181579,-0.0239085,-0.118443,-0.387647,-0.432678,-0.32265,-0.175959,0.0511242,-0.0657003,0.0282204,-0.483335
11625,-0.14636,-0.104615,-0.242208,-0.0680301,0.268976,-0.149826,0.108663,-0.199959,0.0126483,0.160107,0.0637124
12743,-0.121988,0.0384331,0.14049,0.117804,0.432167,0.381956,0.211245,0.007785,0.125206,0.169535,-0.0687908


In [60]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ALAt2r: ala__L_e + h_e <=> ala__L_c + h_c 12743 or 14229 or 15074 or 8962 or 9319 or 9322 or 9962
ARGt2r: arg__L_e + h_e <=> arg__L_c + h_c 14229 or 15074 or 8962 or 9319 or 9322 or (14229 and 8962) or (14229 and 9319) or (14229 and 9322) or (15074 and 8962) or (15074 and 9319) or (15074 and 9322)
ASNt2r: asn__L_e + h_e <=> asn__L_c + h_c 14229 or 15074 or 8962 or 9319 or 9322 or 9962 or (14229 and 8962) or (14229 and 9319) or (14229 and 9322) or (14229 and 9962) or (15074 and 8962) or (15074 and 9319) or (15074 and 9322) or (15074 and 9962)
ASPt2r: asp__L_e + h_e <=> asp__L_c + h_c 14229 or 15074 or 8962 or 9319 or 9322 or 9962
GLNt2r: gln__L_e + h_e <=> gln__L_c + h_c 14229 or 15074 or 8962 or 9319 or 9322 or 9962 or (14229 and 8962) or (14229 and 9319) or (14229 and 9322) or (14229 and 9962) or (15074 and 8962) or (15074 and 9319) or (15074 and 9322) or (15074 and 9962)
GLUt2r: glu__L_e + h_e <=> glu__L_c + h_c 14229 or 15074 or 8962 or 9319 or 9322 or 9962
GLYt2r: gly_e + h_e <=> g

In [61]:
# 14229 and 15074 GAP1 general amino acid permease
# 8962, 9319, 9322, 9962 DIP5 dicarboxylic aa permease glu, asp, gln, asn, ser, ala, gly
# 11625 SLC7A11 Sodium-independent, high-affinity exchange of anionic amino acids with 
#       high specificity for anionic form of cystine and glutamate -> orthoMCL found different SLC7?
# 9135 is oligo-1,6-glucosidase, remove them from transport reactions
# 14129 SLC38A3/SLC38A11 sodium-coupled neutral amino acid transporter, ok
# 11269 UGA4 gamma aminobutyrate permease putrescine, delta-aminolevulinic acid
# 13423 TPO5 secretion of putrescine, spermidine
# 15991 and 16258 HNM1 plasma membrane transporter for choline, ethanolamine, and carnitine, ok
# 16187 vacuolar import large neutral aa
# 12743 vacuolar export large neutral aa
# 8766 vacuolar export asp and glu
# 11017, 13128, 11649, 14774 TPO1-4 polyamine MFS transporter
# 16454 YHK8 presumed MFS drug:H antiporter

model.reactions.get_by_id('AMY1e').gene_reaction_rule = '9135'
model.reactions.get_by_id('AMY2e').gene_reaction_rule = '9135'
model.reactions.get_by_id('CYSTSERex').gene_reaction_rule = '11625'
model.reactions.get_by_id('SERLYSNaex').gene_reaction_rule = '11625'

model.reactions.get_by_id('ABUTt2r').gene_reaction_rule = '11269'
model.reactions.get_by_id('5AOPt2').gene_reaction_rule = '11269'
model.reactions.get_by_id('SPMDtex2').gene_reaction_rule = '13423'
model.reactions.get_by_id('PTRCtex2').gene_reaction_rule = '13423'

model.reactions.get_by_id('PTRCt3i').gene_reaction_rule = '11017 or 11649 or 13128 or 14774'
model.reactions.get_by_id('SPMDt3i').gene_reaction_rule = '11017 or 11649 or 13128 or 14774'
model.reactions.get_by_id('SPRMt2i').gene_reaction_rule = '11017 or 11649 or 13128 or 14774'

model.reactions.get_by_id('ALAt2r').gene_reaction_rule = '14229 or 15074 or 8962 or 9319 or 9322 or 9962'
model.reactions.get_by_id('GLNt2r').gene_reaction_rule = '14229 or 15074 or 8962 or 9319 or 9322 or 9962'
model.reactions.get_by_id('ASNt2r').gene_reaction_rule = '14229 or 15074 or 8962 or 9319 or 9322 or 9962'
model.reactions.get_by_id('ASPt2r').gene_reaction_rule = '14229 or 15074 or 8962 or 9319 or 9322 or 9962'
model.reactions.get_by_id('SERt2r').gene_reaction_rule = '14229 or 15074 or 8962 or 9319 or 9322 or 9962'
model.reactions.get_by_id('GLUt2r').gene_reaction_rule = '14229 or 15074 or 8962 or 9319 or 9322 or 9962'
model.reactions.get_by_id('GLYt2r').gene_reaction_rule = '14229 or 15074 or 8962 or 9319 or 9322 or 9962'

model.reactions.get_by_id('ARGt2r').gene_reaction_rule = '14229 or 15074'
model.reactions.get_by_id('LYSt2r').gene_reaction_rule = '14229 or 15074'
model.reactions.get_by_id('ORNt2r').gene_reaction_rule = '14229 or 15074'
model.reactions.get_by_id('PROt2r').gene_reaction_rule = '14229 or 15074'
model.reactions.get_by_id('DALAt2r').gene_reaction_rule = '14229 or 15074'
model.reactions.get_by_id('PRODt2r').gene_reaction_rule = '14229 or 15074'

remove = ['GLYVESSEC','BALAVECSEC','GABAVESSEC','4ABUTtmi','ASPt2m','GLUt2m','ASNtx','ASPtx']
model.remove_reactions(remove ,remove_orphans=True)

In [62]:
for r in sorted(model.metabolites.get_by_id('val__L_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('leu__L_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('ile__L_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

BIOMASS_RT: 1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024 dgmp_c + 0.0036 dtmp_c + 0.0007 ergst_r + 0.1054 gln__L_c + 0.3018 glu__L_c + 0.2904 gly_c + 0.5185 glycogen_c + 0.046 gmp_c + 59.276 h2o_c + 0.0663 his__L_c + 0.1927 ile__L_c + 0.2964 leu__L_c + 0.2862 lys__L_c + 0.8079 mannan_r + 0.0507 met__L_c + 6e-06 pa_RT_r + 6e-05 pc_RT_r + 4.5e-05 pe_RT_r + 0.1339 phe__L_c + 0.1647 pro__L_c + 1.7e-05 ps_RT_r + 5.3e-05 ptd1ino_RT_r + 0.00099 ribflv_c + 0.1854 ser__L_c + 0.02 so4_c + 0.1914 thr__L_c + 0.0234 tre_c + 6.6e-05 triglyc_RT_r + 0.0284 trp__L_c + 0.102 tyr__L_c + 0.0599 ump_c + 0.2646 val__L_c + 0.0015 zymst_r --> 59.276 adp_c + 58.70001 h_c + 59.305 pi_c 
VALTA: akg_c + val__L_c <=> 3mob_c + glu__L_c 14014 or 14610 or 14853
VALTRS: atp_c + trnaval_c + val__L_c --> amp_c + ppi_c + valtrna_c 16528
VALt2r: h_e + val__L_e <=> h_c + val__L

In [63]:
for r in sorted(model.metabolites.get_by_id('val__L_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('leu__L_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('ile__L_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

VALTAim: akg_m + val__L_m --> 3mob_m + glu__L_m 14014 or 14610 or 14853
VALTRSm: atp_m + trnaval_m + val__L_m --> amp_m + ppi_m + valtrna_m 16528

LEUTAm: akg_m + leu__L_m <=> 4mop_m + glu__L_m 14014 or 14610 or 14853
LEUTRSm: atp_m + leu__L_m + trnaleu_m --> amp_m + leutrna_m + ppi_m 10144

ILETAm: akg_m + ile__L_m <=> 3mop_m + glu__L_m 14014 or 14610 or 14853
ILETRSm: atp_m + ile__L_m + trnaile_m --> amp_m + iletrna_m + ppi_m 13446 or 15556


In [64]:
temp = ['16528','10144','13446','15556']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
16528,"K01873: VARS, valS; valyl-tRNA synthetase",S,VAS1,"VARS,VARS2",VAS1,VARS,Essential,"mito 15.5, cyto_mito 11.833, cyto 7, cyto_nucl...",LRV*
10144,"K01869: LARS, leuS; leucyl-tRNA synthetase",,CDC60,LARS,CDC60,LARS,Essential,"cyto 13.5, cyto_mito 11, mito 7.5, nucl 4",YNI*
13446,"K01870: IARS, ileS; isoleucyl-tRNA synthetase",,ILS1,IARS,ILS1,IARS,Essential,"cyto_nucl 11, nucl 10.5, cyto 10.5, mito 3",VWA*
15556,"K01870: IARS, ileS; isoleucyl-tRNA synthetase",,ISM1,IARS2,ISM1,IARS2,Essential,"mito 20, cyto 4, nucl 3",VSA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,


In [65]:
# 13446 cyto 15556 mito
model.reactions.get_by_id('ILETRS').gene_reaction_rule = '13446'
model.reactions.get_by_id('ILETRSm').gene_reaction_rule = '15556'

In [66]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('aminotransferase')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8540,K14264: BNA3; kynurenine aminotransferase,,BNA3,"KYAT3,KYAT1",BNA3,KYAT3,Not Essential,"cyto 17, cyto_nucl 11, pero 4, nucl 3",RDA*
8936,"K14454: GOT1; aspartate aminotransferase, cyto...",,AAT2,GOT1,AAT2,GOT2,Not Essential,"cyto 18.5, cyto_nucl 12.5, nucl 3.5, pero 3",GQL*
10152,"KOG0257: Kynurenine aminotransferase, glutamin...",,,,,KYAT3,Not Essential,"cyto 11.5, cyto_nucl 10.833, cyto_pero 9.166, ...",LKK*
10937,K07250: gabT; 4-aminobutyrate aminotransferase...,,,AGXT2,ARG8,AGXT2,Not Essential,"cyto 15, cyto_mito 12.333, cyto_nucl 8.833, mi...",HEG*
12099,"K00820: glmS, GFPT; glucosamine--fructose-6-ph...",,"GFA1,YMR085W","GFPT1,GFPT2",GFA1,GFPT2,Essential,"cysk 12, cyto 6.5, mito 6, cyto_nucl 5",TVE*
12407,"K00825: AADAT, KAT2; kynurenine/2-aminoadipate...",,ARO8,AADAT,ARO8,AADAT,Not Essential,"cyto 11.5, cyto_nucl 11.5, nucl 8.5, pero 6",EKA*
12731,K19562: BIO3-BIO1; bifunctional dethiobiotin s...,,,,BIO3,,Essential,"mito 11, extr 10, cyto 3, pero 2",GAS*
13230,K00817: hisC; histidinol-phosphate aminotransf...,,HIS5,,HIS5,,Essential,"mito 13.5, cyto_mito 12.333, cyto 10, cyto_nuc...",FLQ*
13930,"KOG0257: Kynurenine aminotransferase, glutamin...",,,,BNA3,TAT,Not Essential,"cyto 10.5, cyto_nucl 8, mito 6, nucl 4.5, pero 4",LQE*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
8540,-0.157945,-0.017177,-0.0517655,-0.107362,0.311773,0.0292196,0.160261,0.186308,-0.0791432,-0.421139,0.0482614
8936,-0.354295,-0.277708,-0.923495,-1.45071,-4.24826,-4.247,-0.440484,-0.917334,-0.37578,0.0703331,-0.272326
10152,-0.028338,-0.135224,-0.0818158,-0.123987,0.0632719,-0.0688348,-0.0579668,0.600573,-0.00597398,-0.188567,0.396747
10937,0.125487,0.362162,0.242396,0.177756,0.0130856,0.391006,-0.254579,0.281721,-0.244334,-0.480414,-0.324646
12407,-0.120884,0.0459414,-0.0417789,-0.263358,-0.162155,-0.400014,0.275532,0.206367,0.0858757,0.335526,1.09564
13930,-0.35523,-0.34978,-0.178866,-0.644413,-0.310697,-0.499722,0.290919,-0.0131707,-0.21765,-0.179061,-0.0730051
14014,0.277808,0.132282,0.304391,0.220584,0.430317,0.603165,-0.337377,-0.942644,-0.236419,-0.266986,-0.316429
14281,-0.134443,0.0025888,-0.314789,-0.228214,-0.0206698,-0.461234,-0.785788,-2.1105,-0.415618,-0.136043,-0.195287
14610,-0.00277608,-0.0846046,-0.813033,-0.269283,0.667987,-0.430488,-0.572899,-1.45413,0.0684007,0.163426,0.132251


In [67]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

3HKYNAKGAT: akg_c + hLkynr_c --> 42A3HP24DB_c + glu__L_c 12407 or 14908 or 15839 or 8540
FKYNH: Lfmkynr_c + h2o_c --> Lkynr_c + for_c + h_c 8540
KYNAKGAT: Lkynr_c + akg_c --> 4aphdob_c + glu__L_c 12407 or 14908 or 15839 or 8540

3SALATAi: 3sala_c + akg_c + h_c --> 3snpyr_c + glu__L_c 8936
ASPTA: akg_c + asp__L_c <=> glu__L_c + oaa_c 14281 or 8936
ASPTAp: akg_x + asp__L_x <=> glu__L_x + oaa_x 8936
CYSTA: akg_c + cys__L_c --> glu__L_c + mercppyr_c 14281 or 8936
DLYSOXGAT: akg_c + lys__D_c <=> 6a2ohxnt_c + glu__D_c 8936
DLYSPYRAT: lys__D_c + pyr_c <=> 6a2ohxnt_c + ala__D_c 8936
EHGLAT: akg_c + e4hglu_c --> 4h2oglt_c + glu__L_c 8936
EHGLATp: akg_x + e4hglu_x --> 4h2oglt_x + glu__L_x 8936
LCYSTAT: Lcyst_c + akg_c <=> 3spyr_c + glu__L_c 8936
LEUTAi: 4mop_c + glu__L_c --> akg_c + leu__L_c 14610 or 14853 or 8936
PHETA1: akg_c + phe__L_c <=> glu__L_c + phpyr_c 12407 or 13230 or 14281 or 14610 or 14853 or 14908 or 15839 or 8936
TYRTA: akg_c + tyr__L_c <=> 34hpp_c + glu__L_c 12407 or 13230 or 142

gene annotation and blast search  
12407 (tryptophan), 13230 (histidinol-phosphate), 14281 (aspartate, mito), 8936 (aspartate, cyto)  
14908, 15839 (aromatic, phenylalanine/tyrosine/3-methylphenylalanine, 2-aminoadipate, tryptophan, tyrosine)  
16065 TAT (tyrosine)  

In [68]:
# 8540 BNA3 (kynurenine, cyto), also catalyzed by ARO8 and ARO9
# https://www.nature.com/articles/s41598-017-12392-6
# 12407 is for tryptophan by KEGG matches, 14908 and 15839 are ARO8
model.reactions.get_by_id('3HKYNAKGAT').gene_reaction_rule = '14908 or 15839 or 8540'
model.reactions.get_by_id('KYNAKGAT').gene_reaction_rule = '14908 or 15839 or 8540'
# KYNAKGAT and 3HKYNAKGAT needs following spontaneous reactions 'KYNATESYN','r0647'
for x in ['KYNATESYN','r0647']:
    r = hsa2.reactions.get_by_id(x).copy()
    model.add_reactions([r])
# FKYNH is 14788 BNA7, not BNA3 (iMM904 and metacyc wrong)
model.reactions.get_by_id('FKYNH').gene_reaction_rule = '14788'
# 8936 (aspartate, cyto), 14281 (aspartate, mito)
model.reactions.get_by_id('ASPTA').gene_reaction_rule = '8936'
model.reactions.get_by_id('CYSTA').gene_reaction_rule = '8936'
model.reactions.get_by_id('CYSTA').lower_bound = -1000.0
# 3SALATAi is blocked, need to add 3SPYRSP and 3SPYRSPm (spontaneous) to degrade 3snpyr to pyr and so3  
# 3sala is 3-sulfinoalanine / L-cysteine Sulfinic acid in biolog
# In metacyc, 3sala is degraded to Lcyst spontaneously using nad, but 3SALAOX is using O2 
# Lcyst can be converted to taurine, and taurine is degraded to aacald, succ, and so3
# aacald needs to be secreted out, DM_aacald_c is sink in iML1515
for x in ['3SPYRSP','3SPYRSPm','3SALAOX']:
    r = hsa2.reactions.get_by_id(x).copy()
    r.gene_reaction_rule = '12061'
    model.add_reactions([r])
for x in ['DM_aacald_c']:
    r = eco.reactions.get_by_id(x).copy()
    model.add_reactions([r])
# DLYSOXGAT, DLYSPYRAT, EHGLAT, EHGLATp are blocked - remove
# CYSTAm is correct remove CYSATm (wrong)
# EHGLATm/EHGLAT2m leads to 4-hydroxy-2-oxoglutarate, which can be degraded by 12061 HOGA1 (mito in uniprot)
# Add DDPGAm from hsa2 and change gene to 12061
for x in ['DDPGAm']:
    r = hsa2.reactions.get_by_id(x).copy()
    r.gene_reaction_rule = '12061'
    model.add_reactions([r])
# 14610, 14853 (branched, leu/val/ile, mito), 14014 (branched, cyto)
# Remove LEUTAi
model.reactions.get_by_id('VALTA').gene_reaction_rule = '14014'
model.reactions.get_by_id('LEUTA').gene_reaction_rule = '14014'
model.reactions.get_by_id('ILETA').gene_reaction_rule = '14014'
model.reactions.get_by_id('VALTAim').gene_reaction_rule = '14610 or 14853'
model.reactions.get_by_id('LEUTAm').gene_reaction_rule = '14610 or 14853'
model.reactions.get_by_id('ILETAm').gene_reaction_rule = '14610 or 14853'
# VALTAim reversible
model.reactions.get_by_id('VALTAim').id = 'VALTAm'
model.reactions.get_by_id('VALTAm').lower_bound = -1000.0
# OMCDC/OMCDCm spontaneous
model.reactions.get_by_id('OMCDC').gene_reaction_rule = ''
model.reactions.get_by_id('OMCDCm').gene_reaction_rule = ''
# TRPTA is by 12407 and ARO8, correct
# TYRTA is by TAT or ARO8/9, remove TYRTAi and TYRTAim
# Keep? needed for 4hb production in pero TYRTAip: 34hpp_x + glu__L_x --> akg_x + tyr__L_x 8936
model.reactions.get_by_id('TYRTA').gene_reaction_rule = '14908 or 15839 or 16065'
# TYRTA makes 34hpp_c, where is it going?
# PHETA1 is by ARO8, remove PHETA1m
model.reactions.get_by_id('PHETA1').gene_reaction_rule = '14908 or 15839'
# AATA is by AADAT and ARO8
model.reactions.get_by_id('AATA').gene_reaction_rule = '14908 or 15839'
# UNK3 is unknown 2-oxo-4-methylthiobutanoate-glutamine aminotransferase 
# Known e. coli gene ybdL blast to 8540 BNA3 with 37% id, A. thaliana BCAT4 blast to 14014 cyto BCAT with 39.5% id
# BCAT makes sense since it is active on oxo substrates
model.reactions.get_by_id('UNK3').gene_reaction_rule = '8540 or 14014'
# 13930 (unclear, similar to Sulfolobus AspAT, aspartate/aromatic or kynurenine or arginine-pyruvate)  
# 10152 (unclear, similar to Sulfolobus AspAT, and eukaryotic bifunctional aspartate/prephenate)

remove = ['DLYSOXGAT','DLYSPYRAT','EHGLAT','EHGLATp','CYSATm','LEUTAi','TYRTAi','TYRTAim','PHETA1m']
model.remove_reactions(remove, remove_orphans=True)

In [69]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('aminotransferase')]
display(Annotation.loc[temp])

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8540,K14264: BNA3; kynurenine aminotransferase,,BNA3,"KYAT3,KYAT1",BNA3,KYAT3,Not Essential,"cyto 17, cyto_nucl 11, pero 4, nucl 3",RDA*
8936,"K14454: GOT1; aspartate aminotransferase, cyto...",,AAT2,GOT1,AAT2,GOT2,Not Essential,"cyto 18.5, cyto_nucl 12.5, nucl 3.5, pero 3",GQL*
10152,"KOG0257: Kynurenine aminotransferase, glutamin...",,,,,KYAT3,Not Essential,"cyto 11.5, cyto_nucl 10.833, cyto_pero 9.166, ...",LKK*
10937,K07250: gabT; 4-aminobutyrate aminotransferase...,,,AGXT2,ARG8,AGXT2,Not Essential,"cyto 15, cyto_mito 12.333, cyto_nucl 8.833, mi...",HEG*
12099,"K00820: glmS, GFPT; glucosamine--fructose-6-ph...",,"GFA1,YMR085W","GFPT1,GFPT2",GFA1,GFPT2,Essential,"cysk 12, cyto 6.5, mito 6, cyto_nucl 5",TVE*
12407,"K00825: AADAT, KAT2; kynurenine/2-aminoadipate...",,ARO8,AADAT,ARO8,AADAT,Not Essential,"cyto 11.5, cyto_nucl 11.5, nucl 8.5, pero 6",EKA*
12731,K19562: BIO3-BIO1; bifunctional dethiobiotin s...,,,,BIO3,,Essential,"mito 11, extr 10, cyto 3, pero 2",GAS*
13230,K00817: hisC; histidinol-phosphate aminotransf...,,HIS5,,HIS5,,Essential,"mito 13.5, cyto_mito 12.333, cyto 10, cyto_nuc...",FLQ*
13930,"KOG0257: Kynurenine aminotransferase, glutamin...",,,,BNA3,TAT,Not Essential,"cyto 10.5, cyto_nucl 8, mito 6, nucl 4.5, pero 4",LQE*


In [70]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

3HKYNAKGAT: akg_c + hLkynr_c --> 42A3HP24DB_c + glu__L_c 14908 or 15839 or 8540
KYNAKGAT: Lkynr_c + akg_c --> 4aphdob_c + glu__L_c 14908 or 15839 or 8540
UNK3: 2kmb_c + glu__L_c --> akg_c + met__L_c 8540 or 14014

3SALATAi: 3sala_c + akg_c + h_c --> 3snpyr_c + glu__L_c 8936
ASPTA: akg_c + asp__L_c <=> glu__L_c + oaa_c 8936
ASPTAp: akg_x + asp__L_x <=> glu__L_x + oaa_x 8936
CYSTA: akg_c + cys__L_c <=> glu__L_c + mercppyr_c 8936
LCYSTAT: Lcyst_c + akg_c <=> 3spyr_c + glu__L_c 8936
TYRTAip: 34hpp_x + glu__L_x --> akg_x + tyr__L_x 8936

10152 no reactions

ABTA: 4abut_c + akg_c --> glu__L_c + sucsal_c 10937

GF6PTA: f6p_c + gln__L_c --> gam6p_c + glu__L_c 12099

TRPTA: akg_c + trp__L_c <=> glu__L_c + indpyr_c 12407 or 14908 or 15839

AMAOTrm: 8aonn_m + amet_m <=> amob_m + dann_m 12731
DBTSm: atp_m + co2_m + dann_m <=> adp_m + dtbt_m + 3.0 h_m + pi_m 12731

HSTPT: glu__L_c + imacp_c --> akg_c + hisp_c 13230

13930 no reactions

ILETA: akg_c + ile__L_c <=> 3mop_c + glu__L_c 14014
LEUTA: akg_

TYRTA generates 3-(4-Hydroxyphenyl)pyruvate, but it is blocked

11840 is mito 4-hydroxyphenylpyruvate dioxygenase  
Add 34HPPOR from sce, and change genes to '11840'  
Next step is HGNTOR by 15706 mito homogentisate 1,2-dioxygenase
Next step is MACACI by 8431 maleylacetoacetate isomerase  
Add MACACI from sce, and change genes to '8431'  
Next step is FUMAC by 11962  

34HPPOR: 34hpp_c + o2_c → co2_c + hgentis_c 11840 or 9106  
HGNTOR: hgentis_c + o2_c --> 4mlacac_c + h_c 15706  
MACACI: 4mlacac_c --> 4fumacac_c 8431  
FUMAC: 4fumacac_c + h2o_c --> acac_c + fum_c + h_c 11962  

Can Rhodo grow on Tyrosine as a sole carbon source?

3-(4-Hydroxyphenyl)pyruvate can also be decarboxylated by PDC  
Make 34HPPYRDC and add gene 15791 PDC  
PPYRDC: h_c + 34hpp_c --> co2_c + 4hoxpacd_c 15791  
4HOXPACDOX_NADP: 4hoxpacd_c + h2o_c + nadp_c <=> 4hphac_c + 2.0 h_c + nadph_c 12042 or 13426 or 16323  

Many reactions possible from (4-hydroxyphenyl)acetate, check later

In [71]:
r1 = sce.reactions.get_by_id('34HPPOR').copy()
r2 = sce.reactions.get_by_id('MACACI').copy()
r1.gene_reaction_rule = '11840'
r2.gene_reaction_rule = '8431'
model.add_reactions([r1,r2])
model.reactions.get_by_id('PHETA1').gene_reaction_rule = '14908 or 15839'

r = model.reactions.get_by_id('PPYRDC').copy()
r.id = '34HPPYRDC'
r.name = '3-(4-Hydroxyphenyl)pyruvate decarboxylase'
r.gene_reaction_rule = '15791'
model.add_reactions([r])
r.add_metabolites({'phpyr_c': 1.0, 'pacald_c': -1.0, '34hpp_c': -1.0, '4hoxpacd_c': 1.0})

In [72]:
Annotation.loc[['11840','15706','8431','11962']]

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
11840,HMMPfam:Xylose isomerase-like TIM barrel:PF012...,S,,,,,Not Essential,"mito 9.5, extr 9, cyto_mito 6.5, cyto 2.5, nuc...",ADD*
15706,"K00451: HGD, hmgA; homogentisate 1,2-dioxygenase",,,HGD,,HGD,Not Essential,"mito 13.5, cyto_mito 11.333, cyto 8, cyto_nucl...",NGH*
8431,"K01800: maiA, GSTZ1; maleylacetoacetate isomerase",,,,,GSTO1,Not Essential,"mito 21.5, cyto_mito 13, cyto 3.5",QKL*
11962,"K01555: FAH, fahA; fumarylacetoacetase",,,FAH,,FAH,Not Essential,"cyto 7.5, mito 6, cyto_nucl 6, cysk 6, extr 3,...",RMA*


Tryptophan can be degraded via indolepyruvate  
TRPTA: akg_c + trp__L_c <=> glu__L_c + indpyr_c 12407 or 14908 or 15839  
INDPYRD: h_c + indpyr_c <=> co2_c + id3acald_c 15791  
ALDD20y: h2o_c + id3acald_c + nadp_c --> 2.0 h_c + ind3ac_c + nadph_c 12042 or 13426  
ALDD20x: h2o_c + id3acald_c + nad_c --> 2.0 h_c + ind3ac_c + nadh_c 12042 or 13426 or 15814 or 16323  
ALCD26xi: h_c + id3acald_c + nadh_c --> ind3eth_c + nad_c 14108 or 14109 or 15438  
ALDD20x: h2o_c + id3acald_c + nad_c --> 2.0 h_c + ind3ac_c + nadh_c 12042 or 13426 or 15814 or 16323  
NTRLASE: 2.0 h2o_c + ind3acnl_c --> ind3ac_c + nh4_c 12410 or 14925 or 15730  
AMID3: h2o_c + iad_c --> ind3ac_c + nh4_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791  
ALCD26xi: h_c + id3acald_c + nadh_c --> ind3eth_c + nad_c 14108 or 14109 or 15438  
Need to get rid of ind3eth  
Add IND3ETHt and EX_ind3eth_e from sce

Tryptophan can also be degraded via kynurenine  
TRPO2: o2_c + trp__L_c --> Lfmkynr_c 15923  
FKYNH: Lfmkynr_c + h2o_c --> Lkynr_c + for_c + h_c 14788  
KYN3OX: Lkynr_c + h_c + nadph_c + o2_c --> h2o_c + hLkynr_c + nadp_c 9267  
HKYNH: h2o_c + hLkynr_c --> 3hanthrn_c + ala__L_c 8725  
3HAO: 3hanthrn_c + o2_c --> cmusa_c + h_c 8602  
PCLAD: cmusa_c + h_c --> am6sa_c + co2_c 12916  
Need to add next steps AM6SAD and AMCOXO are missing  
AM6SAD known genes blast to general aldehyde dehydrogenases (12042 or 13426) with high %id  
AMCOXO looks like a lumped reaction including deaminase (9857) and reductase (unknown)

NAD biosynthesis from 2-amino-3-carboxymuconate semialdehyde (cmusa)  
Add QUILSYN (spontaneous) from sce  
QUILSYN: cmusa_c --> h2o_c + h_c + quln_c  
NNDPR: 2.0 h_c + prpp_c + quln_c --> co2_c + nicrnt_c + ppi_c 10331  
NICRNS: atp_c + nicrns_c --> adp_c + h_c + nicrnt_c 14574  
NAPRT: h_c + nac_c + prpp_c --> nicrnt_c + ppi_c 10292  
NNAT: atp_c + h_c + nicrnt_c --> dnad_c + ppi_c 10430  
NAMNPP: atp_c + h2o_c + nac_c + prpp_c --> adp_c + nicrnt_c + pi_c + ppi_c 10292  
NNDPR: 2.0 h_c + prpp_c + quln_c --> co2_c + nicrnt_c + ppi_c 10331  
DNADDP: dnad_c + h2o_c --> amp_c + 2.0 h_c + nicrnt_c 14638 or 15385  
NADS1: atp_c + dnad_c + nh4_c --> amp_c + h_c + nad_c + ppi_c 9283  
NADS2: atp_c + dnad_c + gln__L_c + h2o_c --> amp_c + glu__L_c + h_c + nad_c + ppi_c 9283

Tryptophan can also be degraded via anthranilate  
KYN: Lkynr_c + h2o_c --> ala__L_c + anth_c + h_c 8725  
ACANTHAT: accoa_c + anth_c --> acanth_c + coa_c 15430  

Tryptophan biosynthesis  
ANS: chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c 15109 and 16564  
ANPRT: anth_c + prpp_c --> ppi_c + pran_c 9900  
PRAIi: pran_c --> 2cpr5p_c 16564  
IGPS: 2cpr5p_c + h_c --> 3ig3p_c + co2_c + h2o_c 16564  
TRPS1: 3ig3p_c + ser__L_c --> g3p_c + h2o_c + trp__L_c 9262  
16564 is anthranilate synthase / indole-3-glycerol phosphate synthase / phosphoribosylanthranilate isomerase  
(three enzyme in one, gene model is wrong at the beginning)

In [73]:
r1 = sce.reactions.get_by_id('IND3ETHt').copy()
r2 = sce.reactions.get_by_id('EX_ind3eth_e').copy()
model.add_reactions([r1,r2])

r1 = hsa.reactions.get_by_id('AM6SAD').copy()
r2 = hsa.reactions.get_by_id('AMCOXO').copy()
r1.gene_reaction_rule = '12042 or 13426'
r2.gene_reaction_rule = '9857'
model.add_reactions([r1,r2])

r = sce.reactions.get_by_id('QUILSYN').copy()
model.add_reactions([r])

#Fix GPR
model.reactions.get_by_id('ANS').gene_reaction_rule = '15109 and 16564'

In [74]:
temp = ['15923','14788','9267','8725','8602','12916','15430','15109','16564','9900','9262','13669']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
15923,"K00463: IDO, INDO; indoleamine 2,3-dioxygenase",,BNA2,"IDO1,IDO2",BNA2,IDO1,Not Essential,"mito 21, cyto 3, nucl 2, cyto_pero 2",QQE*
14788,K14263: BNA7; kynurenine formamidase,,BNA7,,BNA7,,Not Essential,"cyto 15, cyto_nucl 11.833, cyto_mito 9.499, nu...",VRK*
9267,K00486: KMO; kynurenine 3-monooxygenase,S,BNA4,KMO,BNA4,KMO,Not Essential,"cyto 9, extr 6, E.R. 5, mito 2, plas 2, pero 2",RVD*
8725,"K01556: KYNU, kynU; kynureninase",S,BNA5,KYNU,BNA5,KYNU,Not Essential,"mito 12.5, cyto_mito 10, cyto 6.5, nucl 4, extr 3",LRQ*
8602,"K00452: HAAO; 3-hydroxyanthranilate 3,4-dioxyg...",,BNA1,HAAO,BNA1,HAAO,Not Essential,"cyto_nucl 12.833, cyto 12.5, nucl 11, cyto_per...",EAK*
12916,K03392: ACMSD; aminocarboxymuconate-semialdehy...,,,ACMSD,,ACMSD,Not Essential,"cyto 13.5, cyto_nucl 8, cysk 8, mito 3",LEI*
15430,"HMMPfam:N-acetyltransferase:PF00797,SUPERFAMIL...",,,NAT2,,,Not Essential,"extr 13, mito 7, cyto 3.5, pero 3",QFQ*
15109,K01657: trpE; anthranilate synthase component I,S,TRP2,,TRP2,,Essential,"cyto 14, cyto_mito 12.333, mito 9.5, cyto_nucl...",RQA*
16564,K13501: TRP1; anthranilate synthase / indole-3...,,TRP3,,TRP3,,Essential,"cyto 14.5, cyto_nucl 8.5, mito 6, extr 3",KEA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
15923,-1.58327,-0.187605,-0.00698488,-0.0148311,-5.22999,-2.92776,-0.544326,-0.413142,-0.115497,-0.427772,-0.127409
14788,-0.233399,-0.305637,-0.0800108,0.195436,-0.295385,0.0592889,0.0538321,-0.0922441,0.176823,0.0955898,-0.0658581
9267,-1.03103,0.119221,0.0440669,0.267081,-3.3961,-2.78866,-0.16855,-0.540039,0.365575,-0.699194,-0.124923
8725,-1.59329,0.192628,-0.118265,-0.0429016,-5.08803,-3.28412,-0.138058,-0.339698,-0.0896783,-0.755181,-0.0748085
8602,-1.85696,-0.183267,0.140202,0.0318403,-4.53739,-2.88738,-0.566587,-1.12046,-0.465372,-1.1859,0.238405
12916,0.264166,0.0723649,0.043976,0.0056242,1.21605,0.0696509,0.200722,-0.111672,0.480955,0.417124,-0.207886
15430,-0.0495182,0.0447665,-0.357126,-0.188565,0.279091,0.485558,-0.995476,0.160607,0.0361053,0.262345,-0.402873
13669,0.0145212,-0.149855,-0.0347267,0.0314421,0.0858744,0.127857,0.079359,-0.172931,0.00296095,0.0207803,-0.20695


In [75]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

5HTRPDOX: 5htrp_c + o2_c --> 5hoxnfkyn_c 15923
MELATN23DOX: melatn_c + o2_c --> fna5moxam_c 15923
SRTN23OX: o2_c + srtn_c --> f5hoxkyn_c 15923
TRPO2: o2_c + trp__L_c --> Lfmkynr_c 15923

FKYNH: Lfmkynr_c + h2o_c --> Lkynr_c + for_c + h_c 14788

4HBHYOX: 4hbz_c + h_c + nadph_c + o2_c --> 34dhbz_c + h2o_c + nadp_c 9267
KYN3OX: Lkynr_c + h_c + nadph_c + o2_c --> h2o_c + hLkynr_c + nadp_c 9267

HKYNH: h2o_c + hLkynr_c --> 3hanthrn_c + ala__L_c 8725
KYN: Lkynr_c + h2o_c --> ala__L_c + anth_c + h_c 8725
LFORKYNHYD: Lfmkynr_c + h2o_c --> ala__L_c + h_c + nformanth_c 8725

3HAO: 3hanthrn_c + o2_c --> cmusa_c + h_c 8602

PCLAD: cmusa_c + h_c --> am6sa_c + co2_c 12916

ACANTHAT: accoa_c + anth_c --> acanth_c + coa_c 15430

ANS: chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c 15109 and 16564

ADCS: chor_c + gln__L_c --> 4adcho_c + glu__L_c 13669 or (13669 and 16564)
ANS: chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c 15109 and 16564
IGPS: 2cpr5p_c + h_c --> 3ig3p_c + co2_c + h2o_c 16

In [76]:
# 13669 ABZ1 para-aminobenzoate synthetase using glutamine
model.reactions.get_by_id('ADCS').gene_reaction_rule = '13669'
model.remove_reactions(['PABB'],remove_orphans=True)

Valine biosynthesis  
cyto  
KARA1: 23dhmb_c + nadp_c <=> alac__S_c + h_c + nadph_c 9176  
DHAD1: 23dhmb_c --> 3mob_c + h2o_c 14626 or 9689  
VALTA: akg_c + val__L_c <=> 3mob_c + glu__L_c 14014  
mito  
ACLSm: h_m + 2.0 pyr_m --> alac__S_m + co2_m 15685 and 9800  
KARA1im: alac__S_m + h_m + nadph_m --> 23dhmb_m + nadp_m 9176  
DHAD1m: 23dhmb_m --> 3mob_m + h2o_m 14626 or 9689  
VALTAm: akg_m + val__L_m <=> 3mob_m + glu__L_m 14610 or 14853  

Leucine biosynthesis  
cyto  
IPPS: 3mob_c + accoa_c + h2o_c --> 3c3hmp_c + coa_c + h_c 14856 or 15488  
IPPMIb: 2ippm_c + h2o_c <=> 3c3hmp_c 14914  
IPPMIa: 3c2hmp_c <=> 2ippm_c + h2o_c 14914  
IPMD: 3c2hmp_c + nad_c --> 3c4mop_c + h_c + nadh_c 10428 or 13894  
OMCDC: 3c4mop_c + h_c --> 4mop_c + co2_c 14014  
LEUTA: akg_c + leu__L_c <=> 4mop_c + glu__L_c 14014  
mito  
IPPSm: 3mob_m + accoa_m + h2o_m --> 3c3hmp_m + coa_m + h_m 15488  
missing 3c3hmp -> 3c2hmp (IPPMIb and IPPMIa)  
missing 3c2hmp -> 3c4mop (IPMD)  
OMCDCm: 3c4mop_m + h_m --> 4mop_m + co2_m 14610 or 14853  
LEUTAm: akg_m + leu__L_m <=> 4mop_m + glu__L_m 14610 or 14853  

Isoleucine biosynthesis  
cyto  
DHAD2: 23dhmp_c --> 3mop_c + h2o_c 14626 or 9689  
LETA: akg_c + ile__L_c <=> 3mop_c + glu__L_c 14014  
mito
THRD_Lm: thr__L_m --> 2obut_m + nh4_m 11909  
ACHBSm: 2obut_m + h_m + pyr_m --> 2ahbut_m + co2_m 15685 and 9800  
KARA2im: 2ahbut_m + h_m + nadph_m --> 23dhmp_m + nadp_m 9176  
DHAD2m: 23dhmp_m --> 3mop_m + h2o_m 14626 or 9689  
ILETAm: akg_m + ile__L_m <=> 3mop_m + glu__L_m 14610 or 14853  
ok, need to make threonine  

In [77]:
temp = ['15685','9800','9176','14626','9689','14014','14610','14853',
        '14856','15488','14914','10428','13894']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
15685,"K01652: E2.2.1.6L, ilvB, ilvG, ilvI; acetolact...",S,ILV2,,ILV2,HACL1,Essential,"mito 26.5, cyto_mito 14",FAK*
9800,"K01653: E2.2.1.6S, ilvH, ilvN; acetolactate sy...",S,ILV6,,ILV6,,Not Essential,mito 26,PPG*
9176,K00053: ilvC; ketol-acid reductoisomerase,,ILV5,,ILV5,,Essential,"mito 24.5, cyto_mito 14",KNN*
14626,K01687: ilvD; dihydroxy-acid dehydratase,,ILV3,,ILV3,,Not Essential,"cyto 21, cyto_nucl 13.5, nucl 4",YTD*
9689,K01687: ilvD; dihydroxy-acid dehydratase,,ILV3,,ILV3,,Essential,"cyto 23.5, cyto_nucl 14",DGP*
14014,"K00826: E2.6.1.42, ilvE; branched-chain amino ...",,"BAT1,BAT2",BCAT1,BAT2,BCAT1,Not Essential,"cyto 12, mito 11, cyto_nucl 6.833, cyto_pero 6...",VVV*
14610,"K00826: E2.6.1.42, ilvE; branched-chain amino ...",S,"BAT1,BAT2",BCAT1,BAT1,BCAT2,Not Essential,"mito 20, cyto 2, E.R. 2, plas 1, extr 1, cyto_...",VDS*
14853,"K00826: E2.6.1.42, ilvE; branched-chain amino ...",S,"BAT1,BAT2",BCAT1,BAT2,BCAT2,Not Essential,"mito 8.5, cyto_mito 7.833, pero 7, cyto 6, ext...",SKA*
14856,"K01655: LYS21, LYS20; homocitrate synthase",,"LYS21,LYS20",,LYS20,,Not Essential,"cyto 17.5, cyto_nucl 15.5, nucl 8.5",TSA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
14626,-0.0945652,-0.537756,-0.266318,-0.0308197,-0.122558,-0.179321,-0.354095,0.163809,-0.20504,0.172839,-0.00261887
9689,0.196183,-0.0642164,0.0647096,0.198793,-0.269961,-0.509114,0.343713,0.642502,-0.236652,-0.991276,-0.694307
14014,0.277808,0.132282,0.304391,0.220584,0.430317,0.603165,-0.337377,-0.942644,-0.236419,-0.266986,-0.316429
14610,-0.00277608,-0.0846046,-0.813033,-0.269283,0.667987,-0.430488,-0.572899,-1.45413,0.0684007,0.163426,0.132251
14853,-0.137609,-0.301241,-0.25221,0.0016244,0.0216384,-0.339282,-0.120204,0.862746,-0.101846,0.237236,-0.385014
14856,-2.41996,-2.32883,-3.21568,-1.90593,-3.79053,-1.27159,-2.91199,-2.78208,-3.91013,-1.46531,-0.724535
10428,-0.412945,0.364408,-1.37317,-0.878157,-0.883249,-0.442528,-1.48324,-2.05839,-2.52439,-0.284016,-0.218092


In [78]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ACHBSm: 2obut_m + h_m + pyr_m --> 2ahbut_m + co2_m 15685 and 9800
ACLSm: h_m + 2.0 pyr_m --> alac__S_m + co2_m 15685 and 9800

ACHBSm: 2obut_m + h_m + pyr_m --> 2ahbut_m + co2_m 15685 and 9800
ACLSm: h_m + 2.0 pyr_m --> alac__S_m + co2_m 15685 and 9800

DPR: 2dhp_c + h_c + nadph_c --> nadp_c + pant__R_c 14277 or 16522 or 9176
DPRm: 2dhp_m + h_m + nadph_m --> nadp_m + pant__R_m 14277 or 9176
KARA1: 23dhmb_c + nadp_c <=> alac__S_c + h_c + nadph_c 9176
KARA1im: alac__S_m + h_m + nadph_m --> 23dhmb_m + nadp_m 9176
KARA2: 2ahbut_c + h_c + nadph_c <=> 23dhmp_c + nadp_c 9176
KARA2im: 2ahbut_m + h_m + nadph_m --> 23dhmp_m + nadp_m 9176

DHAD1: 23dhmb_c --> 3mob_c + h2o_c 14626 or 9689
DHAD1m: 23dhmb_m --> 3mob_m + h2o_m 14626 or 9689
DHAD2: 23dhmp_c --> 3mop_c + h2o_c 14626 or 9689
DHAD2m: 23dhmp_m --> 3mop_m + h2o_m 14626 or 9689

DHAD1: 23dhmb_c --> 3mob_c + h2o_c 14626 or 9689
DHAD1m: 23dhmb_m --> 3mob_m + h2o_m 14626 or 9689
DHAD2: 23dhmp_c --> 3mop_c + h2o_c 14626 or 9689
DHAD2m: 23dhmp_m

In [79]:
# 9176 ILV5 mito, 14626/9689 ILV3 says cyto but ILV3 mito in S. cer
model.remove_reactions(['KARA1','KARA2','DHAD1','DHAD2'],remove_orphans=True)
# 14856 is for lysine and 15488 LEU4 in both cyto and mito in S. cer
model.reactions.get_by_id('IPPS').gene_reaction_rule = '15488'
# 10428 is for lysine, MALDDH in E. coli different gene
model.reactions.get_by_id('IPMD').gene_reaction_rule = '13894'
model.remove_reactions(['MALDDH'],remove_orphans=True)

phosphopantothenate biosynthesis  
2DHPFALDL: 2dhp_c <=> 3mob_c + fald_c 8443  
MOHMT: 3mob_c + h2o_c + mlthf_c --> 2dhp_c + thf_c 8443  
DPR: 2dhp_c + h_c + nadph_c --> nadp_c + pant__R_c 14277 or 16522 or 9176  
PANTS: ala_B_c + atp_c + pant__R_c --> amp_c + h_c + pnto__R_c + ppi_c 10475  

MTMOHT: 3mob_m + h2o_m + mlthf_m --> 2dhp_m + thf_m 8443  
DPRm: 2dhp_m + h_m + nadph_m --> nadp_m + pant__R_m 14277 or 9176  

In [80]:
temp = ['8443','14277','16522','10475']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8443,K00606: panB; 3-methyl-2-oxobutanoate hydroxym...,,ECM31,,ECM31,,Not Essential,"cyto 18, mito 6, nucl 2",REA*
14277,"K00077: panE, apbA; 2-dehydropantoate 2-reductase",,YHR063C,,,,Not Essential,"cyto_nucl 14, nucl 13, cyto 11",SSR*
16522,HMMPfam:Ketopantoate reductase PanE/ApbA:PF025...,S,YDL144C,,YDL144C,,Not Essential,"extr 13, plas 5, cyto 4, E.R. 3",KSA*
10475,K01918: panC; pantoate--beta-alanine ligase,,PAN6,,PAN6,,Not Essential,"cyto 11.5, cyto_mito 10.5, mito 8.5, nucl 4",NPQ*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
8443,-0.0514645,-0.0487158,0.0612808,-0.113505,-0.0411842,-0.480022,0.28359,0.201074,0.460733,0.427401,0.0130797
16522,-0.110156,-0.221079,0.00235083,-0.151444,-0.380114,-0.212064,0.137293,-0.179819,0.0525662,-0.117143,0.0325771
10475,0.480227,0.33277,0.617152,0.59239,0.159394,0.27627,0.026824,-0.255013,-0.188392,-0.201929,0.0510207


In [81]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

2DHPFALDL: 2dhp_c <=> 3mob_c + fald_c 8443
MOHMT: 3mob_c + h2o_c + mlthf_c --> 2dhp_c + thf_c 8443
MTMOHT: 3mob_m + h2o_m + mlthf_m --> 2dhp_m + thf_m 8443

DPR: 2dhp_c + h_c + nadph_c --> nadp_c + pant__R_c 14277 or 16522 or 9176
DPRm: 2dhp_m + h_m + nadph_m --> nadp_m + pant__R_m 14277 or 9176
PNORn: nadp_n + pant__R_n <=> 2dhp_n + h_n + nadph_n 14277

DPR: 2dhp_c + h_c + nadph_c --> nadp_c + pant__R_c 14277 or 16522 or 9176

PANTS: ala_B_c + atp_c + pant__R_c --> amp_c + h_c + pnto__R_c + ppi_c 10475



In [82]:
# 8443 cyto
# 14277 cyto and 16522 is ?, it is not known that 9716 ILV5 has DPR activity like ilvC 
model.reactions.get_by_id('DPR').gene_reaction_rule = '14277 or 16522'
model.remove_reactions(['MTMOHT','2DHPFALDL','DPRm','PNORn'],remove_orphans=True)

In [83]:
temp = ['15773','16581']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
15773,KOG1589: Uncharacterized conserved protein,,"MPC3,MPC2",MPC2,MPC3,MPC2,Not Essential,"mito 13.5, cyto_mito 11.333, cyto 8, cyto_nucl...",LAA*
16581,KOG1590: Uncharacterized conserved protein,,MPC1,"MPC1,MPC1L",MPC1,MPC1,Not Essential,"mito 21, cyto 3, nucl 1, plas 1, pero 1",ALK*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16581,-1.05737,1.06236,-0.0930612,-1.09979,0.0681052,-0.245449,0.297283,-2.5247,0.690153,0.325869,0.53652


In [84]:
r = sce.reactions.get_by_id('PYRt2m').copy()
r.gene_reaction_rule = '15773 and 16581'
model.add_reactions([r])

for x in ['LEUt5m','VALt5m','ILEtmi']:
    r = hsa.reactions.get_by_id(x).copy()
    r.gene_reaction_rule = ''
    model.add_reactions([r])

# Add keto acid transporters
# https://mmbr.asm.org/content/67/1/1
# proton symport reactions in RECON1/3D - generate proton gradient when transporting keto acids out
# makes it better to make amino acids in cyto, but mito BAT1 is for biosynthesis and cyto BAT2 is for degradation
# Mito carriers are usually exchange mechanism, but it is not known -> just add diffusion like transport
for x in ['3MOPtm','3MOBtm']:
    r = sce.reactions.get_by_id(x).copy()
    r.gene_reaction_rule = ''
    model.add_reactions([r])

r = model.reactions.get_by_id('3MOPtm').copy()
r.id = '4MOPtm'
r.name = '4-Methyl-2-oxopentanoate transport diffusion mitochondrial'
model.add_reactions([r])
r.add_metabolites({'3mop_c': 1.0, '3mop_m': -1.0, '4mop_c': -1.0, '4mop_m': 1.0})

In [85]:
sol = model.optimize()
print(sol.objective_value)

for k, v in sol.shadow_prices.items():
    if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
        print(k, v)

0.0
ser__L_c -0.8858729835316214
leu__L_c -0.8858729835316214
dtmp_c -0.8858729835316214
trp__L_c -0.8858729835316214
ile__L_c -0.8858729835316214
arg__L_c -0.8858729835316214
val__L_c -0.8858729835316214
ribflv_c 2.657618950594864


In [86]:
with model:
    r = model.reactions.get_by_id('EX_h_e').copy()
    r.id = 'EX_temp_e'
    model.add_reactions([r])
    r.add_metabolites({'h_e': 1.0, 'thr__L_m': -1.0})
    model.objective = 'EX_temp_e'
    sol = model.optimize()
    print(sol.objective_value)
    r.add_metabolites({'thr__L_m': 1.0, 'thr__L_c': -1.0})
    model.objective = 'EX_temp_e'
    sol = model.optimize()
    print(sol.objective_value)

0.0
14.999999999999996


In [87]:
# ile_L comes from thr_L
for r in sorted(model.metabolites.get_by_id('thr__L_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('thr__L_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

BIOMASS_RT: 1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024 dgmp_c + 0.0036 dtmp_c + 0.0007 ergst_r + 0.1054 gln__L_c + 0.3018 glu__L_c + 0.2904 gly_c + 0.5185 glycogen_c + 0.046 gmp_c + 59.276 h2o_c + 0.0663 his__L_c + 0.1927 ile__L_c + 0.2964 leu__L_c + 0.2862 lys__L_c + 0.8079 mannan_r + 0.0507 met__L_c + 6e-06 pa_RT_r + 6e-05 pc_RT_r + 4.5e-05 pe_RT_r + 0.1339 phe__L_c + 0.1647 pro__L_c + 1.7e-05 ps_RT_r + 5.3e-05 ptd1ino_RT_r + 0.00099 ribflv_c + 0.1854 ser__L_c + 0.02 so4_c + 0.1914 thr__L_c + 0.0234 tre_c + 6.6e-05 triglyc_RT_r + 0.0284 trp__L_c + 0.102 tyr__L_c + 0.0599 ump_c + 0.2646 val__L_c + 0.0015 zymst_r --> 59.276 adp_c + 58.70001 h_c + 59.305 pi_c 
THRA: thr__L_c --> acald_c + gly_c 16182
THRD_L: thr__L_c --> 2obut_c + nh4_c 11909 or 9216
THRS: h2o_c + phom_c --> pi_c + thr__L_c 9742
THRTRS: atp_c + thr__L_c + trnathr_c --> amp

In [88]:
temp = ['9742','11909','9216','11849']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
9742,K01733: thrC; threonine synthase,,THR4,THNSL2,THR4,THNSL,Essential,"cyto 18, cyto_nucl 12.5, nucl 5, pero 2, cysk 2",SGL*
11909,"K01754: E4.3.1.19, ilvA, tdcB; threonine dehyd...",,ILV1,,ILV1,SRR,Not Essential,"cyto 13.5, cyto_nucl 9.5, mito 7, nucl 4.5",DAE*
9216,"K17989: SDS, SDH, CHA1; L-serine/L-threonine a...",,"CHA1,YIL167W","SDS,SDSL",SDL1,SDS,Not Essential,"mito 12.5, cyto_mito 12.5, cyto 11.5",VEA*
11849,"K01752: E4.3.1.17, sdaA, sdaB, tdcG; L-serine ...",,,,,,Not Essential,"mito 26.5, cyto_mito 14",PAC*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11909,-0.644152,0.0708412,-0.968321,0.291933,-0.126688,0.609645,-0.56859,-0.287953,-0.564922,-0.208981,-0.120712
9216,-0.0217597,-0.217502,0.0258124,0.0953588,-0.281999,-0.329755,0.728678,-0.168594,0.245015,0.086814,-0.14243
11849,0.0856094,0.0176457,0.0441559,-0.0559005,-0.105301,0.0302815,0.0638251,-0.342133,0.150496,-0.00587411,0.0607802


In [89]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

THRS: h2o_c + phom_c --> pi_c + thr__L_c 9742

SERD_L: ser__L_c --> nh4_c + pyr_c 11849 or 11909 or 9216
SER_AL: ser__L_c <=> h_c + nh3_c + pyr_c 11909
THRD_L: thr__L_c --> 2obut_c + nh4_c 11909 or 9216
THRD_Lm: thr__L_m --> 2obut_m + nh4_m 11909

SERD_L: ser__L_c --> nh4_c + pyr_c 11849 or 11909 or 9216
SERHL: ser__L_c --> 2amac_c + h2o_c 9216
THRD_L: thr__L_c --> 2obut_c + nh4_c 11909 or 9216

SERD_L: ser__L_c --> nh4_c + pyr_c 11849 or 11909 or 9216



In [90]:
# 11909 mito thr
# 9216 cyto CHA1/SDS? ser and thr deaminase, also SERHL and 2AMACHYD
# 11849 mito? cyto? bacterial SDH
model.reactions.get_by_id('SERD_L').gene_reaction_rule = '11849 or 9216'
model.reactions.get_by_id('THRD_L').gene_reaction_rule = '9216'
r = hsa2.reactions.get_by_id('2AMACHYD').copy()
r.gene_reaction_rule = '9216'
model.add_reactions([r])
model.remove_reactions(['SER_AL'],remove_orphans=True)

# Threonine is synthesized in cyto
r = hsa2.reactions.get_by_id('thr_mt').copy()
r.id = 'THRtm'
r.lower_bound = -1000.0
model.add_reactions([r])

# cyto 2obut needs to be either transported out or to mito (known to happen)
r = sce.reactions.get_by_id('2OBUTtm').copy()
r.gene_reaction_rule = ''
model.add_reactions([r])
r = hsa2.reactions.get_by_id('2OBUTt').copy()
r.gene_reaction_rule = '11257'
model.add_reactions([r])
r = hsa2.reactions.get_by_id('EX_2obut_e').copy()
r.lower_bound = 0.0
model.add_reactions([r])

# Need to get rid of nh4_m from THRD_Lm somehow
r = sce.reactions.get_by_id('NH4tm').copy()
# find transporter gene
#r.gene_reaction_rule = ''
model.add_reactions([r])

In [91]:
with model:
    r = model.reactions.get_by_id('EX_h_e').copy()
    r.id = 'EX_temp_e'
    model.add_reactions([r])
    r.add_metabolites({'h_e': 1.0, '2ahbut_m': -1.0})
    model.objective = 'EX_temp_e'
    sol = model.optimize()
    print(sol.objective_value)
    r.add_metabolites({'2ahbut_m': 1.0, '23dhmp_m': -1.0})
    model.objective = 'EX_temp_e'
    sol = model.optimize()
    print(sol.objective_value)

9.999999999999996
0.0


Cannot make nadph_m, need to find a source  
In S. cer, the NADH kinase encoded by the POS5 gene is considered to be the main provider of NADPH in the mitochondrion by phosphorylation of NADH. POS5 also works as NAD kinase, this reaction followed by the dehydrogenase reaction of ALD4/ALD5 is the other source for the synthesis of NADPH. http://www.jbc.org/content/284/12/7553.full  
There is no POS5 in R. toruloides, but cyto NAD+ kinase YEF1/UTR1 10182 exists  

In [92]:
for r in sorted(model.metabolites.get_by_id('nadp_m').reactions, key=lambda x: x.id):
    if 'nadph_m' in [m.id for m in r.products]:
        print(r, r.gene_reaction_rule)
    if 'nadph_m' in [m.id for m in r.reactants] and r.reversibility:
        print(r, r.gene_reaction_rule)

ABOR: 4abutn_m + h2o_m + nadp_m --> 4abut_m + 2.0 h_m + nadph_m 12042 or 13426
ALDD20ym: h2o_m + id3acald_m + nadp_m --> 2.0 h_m + ind3ac_m + nadph_m 12042 or 13426
ALDD2ym: acald_m + h2o_m + nadp_m --> ac_m + 2.0 h_m + nadph_m 12042 or 13426
MTHFDm: mlthf_m + nadp_m <=> methf_m + nadph_m 15366
SACCD4m: h2o_m + nadp_m + saccrp__L_m --> L2aadp6sa_m + glu__L_m + h_m + nadph_m 10577 or 16833


In [93]:
for r in sorted(model.genes.get_by_id('10182').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

NADK: atp_c + nad_c --> adp_c + h_c + nadp_c 10182
NADKm: atp_m + nad_m --> adp_m + h_m + nadp_m 10182


In [94]:
model.remove_reactions(['NADKm'], remove_orphans=True)

In [95]:
for r in sorted(model.genes.get_by_id('15814').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

5HOXINDACTOX: 5hoxindact_c + h2o_c + nad_c --> 5hoxindoa_c + 2.0 h_c + nadh_c 12042 or 13426 or 15814 or 16323
AASAD3: L2aadp6sa_c + h2o_c + nad_c --> L2aadp_c + 2.0 h_c + nadh_c 15814
ALDD20x: h2o_c + id3acald_c + nad_c --> 2.0 h_c + ind3ac_c + nadh_c 12042 or 13426 or 15814 or 16323
ALDD2x: acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c 12042 or 13426 or 15814 or 16323
BAMPPALDOX: bamppald_c + h2o_c + nad_c --> ala_B_c + 2.0 h_c + nadh_c 12042 or 13426 or 15814 or 16323
GCALDD: gcald_c + h2o_c + nad_c --> glyclt_c + 2.0 h_c + nadh_c 12042 or 13426 or 15814 or 16323
IMACTD: h2o_c + im4act_c + nad_c --> 2.0 h_c + im4ac_c + nadh_c 12042 or 13426 or 15814 or 16323
LCADi: h2o_c + lald__L_c + nad_c --> 2.0 h_c + lac__L_c + nadh_c 12042 or 13426 or 15814 or 16323
LCADi_D: h2o_c + lald__D_c + nad_c --> 2.0 h_c + lac__D_c + nadh_c 12042 or 13426 or 15814 or 16323
NABTNO: h2o_c + n4abutn_c + nad_c --> 4aabutn_c + 2.0 h_c + nadh_c 12042 or 13426 or 15814 or 16323
PYLALDOX: h2o_c + nad_c + 

In [96]:
for r in sorted(model.metabolites.get_by_id('acald_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('acald_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

ACALDCD: 2.0 acald_c --> actn__R_c 15791
ALCD2x: etoh_c + nad_c <=> acald_c + h_c + nadh_c 15438
ALCD2y: etoh_c + nadp_c --> acald_c + h_c + nadph_c 11882 or 12784 or 9774
ALDD2x: acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c 12042 or 13426 or 15814 or 16323
ALDD2y: acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + nadph_c 11650 or 12042 or 13426 or 14700 or 16323 or 8666
PYRDC: h_c + pyr_c --> acald_c + co2_c 15791
THRA: thr__L_c --> acald_c + gly_c 16182
THRA2: athr__L_c --> acald_c + gly_c 16182

ALDD2xm: acald_m + h2o_m + nad_m --> ac_m + 2.0 h_m + nadh_m 12042 or 13426 or 16323
ALDD2ym: acald_m + h2o_m + nadp_m --> ac_m + 2.0 h_m + nadph_m 12042 or 13426


In [97]:
temp = ['12042','13426','16323']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
12042,K00128: ALDH; aldehyde dehydrogenase (NAD+),,"ALD5,ALD4,ALD6,ALD3,ALD2","ALDH1A1,ALDH1A2,ALDH1A3,ALDH1B1,ALDH2",ALD5,ALDH2,Not Essential,"cyto 18.5, cyto_nucl 10, mito 4, pero 4",NPL*
13426,K07249: E1.2.1.36; retinal dehydrogenase,,"ALD5,ALD4,ALD6,ALD3,ALD2","ALDH1A1,ALDH1A2,ALDH1A3,ALDH1B1,ALDH2",ALD5,ALDH1,Not Essential,"mito 25.5, cyto_mito 14",WPL*
16323,K00128: ALDH; aldehyde dehydrogenase (NAD+),,HFD1,"ALDH3A1,ALDH3A2,ALDH3B1,ALDH3B2",HFD1,ALDH3,Not Essential,"cyto 10.5, plas 7, cyto_nucl 7, mito 3, nucl 2...",GQA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
12042,0.170666,-0.143043,0.000953096,-0.0230376,-0.521499,0.0146534,0.144875,-0.295284,0.392859,0.454577,-0.242457
13426,-0.226122,-0.181863,-0.169891,-0.0897023,-0.0133507,-0.00809092,-1.1502,-2.84044,-0.85113,-0.458514,-0.0243377
16323,0.000210473,-0.0174318,0.0140589,-0.110443,0.00440814,0.0892768,0.155043,0.103889,-0.177274,-0.243731,-0.140368


In [98]:
temp = ['11650','14700','8666']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
11650,"KOG0023: Alcohol dehydrogenase, class V",,,,ADH2,ADH1B,Not Essential,"cyto 9.5, extr 8, mito 6, cyto_nucl 6, nucl 1.5",VIP*
14700,KOG0725: Reductases with broad range of substr...,,,HSD17B14,YMR226C,DECR2,Not Essential,"cyto 14.5, cyto_nucl 11.5, nucl 5.5, pero 3, c...",YLV*
8666,KOG0725: Reductases with broad range of substr...,,,HSD17B14,SPS19,HSD17B,Not Essential,"cyto 18.5, cyto_nucl 12, nucl 2.5, mito 2, ext...",YVH*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11650,0.162296,-0.125414,0.239373,0.0824604,0.133479,0.152742,0.151596,-0.00688522,-0.0872696,-0.226133,0.148191
14700,-0.136366,0.186102,-0.178815,-0.232998,-0.0364499,-0.0311361,0.24188,-0.184481,-0.138316,0.201217,0.511467
8666,0.246518,-0.0531704,0.304517,-0.024053,-0.100515,-0.173468,-0.0249248,-0.343657,0.409518,0.34975,-0.0161461


In [99]:
with model:
    r = sce.reactions.get_by_id('ACALDtm').copy()
    model.add_reactions([r])
    sol = model.optimize()
    print(sol.objective_value)
    
    for k, v in sol.shadow_prices.items():
        if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
            print(k, v)

0.0
pe_RT_r 124.47099825740602
mannan_r -1.2447099825740602


In [100]:
r = sce.reactions.get_by_id('ACALDtm').copy()
model.add_reactions([r])

In [101]:
for r in sorted(model.metabolites.get_by_id('saccrp__L_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('L2aadp6sa_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('L2aadp_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

SACCD3m: akg_m + h_m + lys__L_m + nadph_m --> h2o_m + nadp_m + saccrp__L_m 10577 or 16833
SACCD4m: h2o_m + nadp_m + saccrp__L_m --> L2aadp6sa_m + glu__L_m + h_m + nadph_m 10577 or 16833

SACCD4m: h2o_m + nadp_m + saccrp__L_m --> L2aadp6sa_m + glu__L_m + h_m + nadph_m 10577 or 16833

2AMADPTm: L2aadp_c + akg_m <=> L2aadp_m + akg_c 10635


In [102]:
temp = ['10577','16833','14087','9495','10220']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10577,K14157: AASS; alpha-aminoadipic semialdehyde s...,,"SPE3,LYS9","SRM,AASS",LYS9,AASS,Not Essential,"mito 21, cyto 3, pero 2",ARF*
16833,K00293: LYS9; saccharopine dehydrogenase (NADP...,,"SPE3,LYS9","SRM,AASS",LYS9,SRMe,Essential,"cyto 19, cysk 6",KVQ*
14087,K00290: LYS1; saccharopine dehydrogenase (NAD+...,,LYS1,,LYS1,,Not Essential,"cyto 16.5, cyto_nucl 10, pero 5, nucl 2.5",KKQ*
9495,K00143: LYS2; L-aminoadipate-semialdehyde dehy...,,LYS2,,LYS2,AASDH,Not Essential,"mito 9, cyto 8.5, cyto_nucl 5.5, extr 5, pero 2",GVV*
10220,"K06133: LYS5, acpT; 4'-phosphopantetheinyl tra...",,,AASDHPPT,,,Not Essential,"nucl 12, cyto_nucl 10, mito 6, cyto 6",AVD*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
10577,-0.0823193,-0.0536589,-0.172111,-0.277613,-0.0621896,-0.131564,-0.239781,-0.0105686,0.20184,0.345546,0.0251315
9495,-0.852305,-0.88796,-1.33062,-0.950872,-1.39312,-1.26513,-1.29749,-1.72025,-1.59537,-1.27751,0.0278676


In [103]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

SACCD1: L2aadp6sa_c + glu__L_c + h_c + nadph_c <=> h2o_c + nadp_c + saccrp__L_c 10577 or 16833
SACCD3m: akg_m + h_m + lys__L_m + nadph_m --> h2o_m + nadp_m + saccrp__L_m 10577 or 16833
SACCD4m: h2o_m + nadp_m + saccrp__L_m --> L2aadp6sa_m + glu__L_m + h_m + nadph_m 10577 or 16833
SPMS: ametam_c + ptrc_c --> 5mta_c + h_c + spmd_c 10577 or 16833
SPRMS: ametam_c + spmd_c --> 5mta_c + h_c + sprm_c 10577 or 16833

SACCD1: L2aadp6sa_c + glu__L_c + h_c + nadph_c <=> h2o_c + nadp_c + saccrp__L_c 10577 or 16833
SACCD3m: akg_m + h_m + lys__L_m + nadph_m --> h2o_m + nadp_m + saccrp__L_m 10577 or 16833
SACCD4m: h2o_m + nadp_m + saccrp__L_m --> L2aadp6sa_m + glu__L_m + h_m + nadph_m 10577 or 16833
SPMS: ametam_c + ptrc_c --> 5mta_c + h_c + spmd_c 10577 or 16833
SPRMS: ametam_c + spmd_c --> 5mta_c + h_c + sprm_c 10577 or 16833

SACCD2: h2o_c + nad_c + saccrp__L_c <=> akg_c + h_c + lys__L_c + nadh_c 14087

AASAD1: L2aadp_c + atp_c + h_c + nadph_c --> L2aadp6sa_c + amp_c + nadp_c + ppi_c 9495 or (1022

In [104]:
# Lysine biosynthesis
temp = ['14856','14600','16016','10428','14908','15839','9495','10220','10577','16833','14087','15814']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
14856,"K01655: LYS21, LYS20; homocitrate synthase",,"LYS21,LYS20",,LYS20,,Not Essential,"cyto 17.5, cyto_nucl 15.5, nucl 8.5",TSA*
14600,K17450: ACO2; homoaconitase,,"ACO2,ACO1","ACO1,IREB2,ACO2",ACO2,ACO2,Not Essential,"mito 20, cyto 4.5, cyto_nucl 3.5",GQA*
16016,K01705: LYS4; homoaconitate hydratase,,LYS4,,LYS4,ACO2,Not Essential,"mito 23, cyto 4",ISA*
10428,K05824: LYS12; homoisocitrate dehydrogenase,,LYS12,,LYS12,IDH3A,Not Essential,"mito 15, cyto 12",KQL*
14908,K00838: ARO8; aromatic amino acid aminotransfe...,,ARO8,AADAT,ARO8,AADAT,Not Essential,"cyto 7, cysk 7, mito 5, cyto_nucl 5, pero 4, m...",WAY*
15839,K00838: ARO8; aromatic amino acid aminotransfe...,,ARO8,AADAT,ARO8,AADAT,Not Essential,"cysk 9, cyto 6, nucl 4, mito 4, pero 4, mito_n...",FKD*
9495,K00143: LYS2; L-aminoadipate-semialdehyde dehy...,,LYS2,,LYS2,AASDH,Not Essential,"mito 9, cyto 8.5, cyto_nucl 5.5, extr 5, pero 2",GVV*
10220,"K06133: LYS5, acpT; 4'-phosphopantetheinyl tra...",,,AASDHPPT,,,Not Essential,"nucl 12, cyto_nucl 10, mito 6, cyto 6",AVD*
10577,K14157: AASS; alpha-aminoadipic semialdehyde s...,,"SPE3,LYS9","SRM,AASS",LYS9,AASS,Not Essential,"mito 21, cyto 3, pero 2",ARF*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
14856,-2.41996,-2.32883,-3.21568,-1.90593,-3.79053,-1.27159,-2.91199,-2.78208,-3.91013,-1.46531,-0.724535
14600,-0.10018,0.289792,-0.141329,0.0684176,-0.00584732,-0.323588,-0.218815,-1.45316,-1.32814,-0.239199,0.545846
16016,-2.28259,-1.40215,-1.81138,-3.40624,-2.91691,-1.86657,-3.35685,-4.04351,-3.50648,-0.684523,0.0693203
10428,-0.412945,0.364408,-1.37317,-0.878157,-0.883249,-0.442528,-1.48324,-2.05839,-2.52439,-0.284016,-0.218092
14908,0.0451271,-0.280956,0.220759,-0.0377905,0.21851,-0.259388,-0.292657,0.206026,0.339453,0.398586,-0.0659921
15839,-0.0847761,-0.12423,0.269542,0.0647239,0.154571,0.146919,-0.346466,-0.214054,-0.177142,-0.229588,-0.0156292
9495,-0.852305,-0.88796,-1.33062,-0.950872,-1.39312,-1.26513,-1.29749,-1.72025,-1.59537,-1.27751,0.0278676
10577,-0.0823193,-0.0536589,-0.172111,-0.277613,-0.0621896,-0.131564,-0.239781,-0.0105686,0.20184,0.345546,0.0251315
15814,-0.464573,-0.6107,-0.102958,-0.543758,-0.425179,-0.212745,-0.360015,-1.03866,0.403738,-0.219486,-0.625104


In [105]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

HCITSn: accoa_n + akg_n + h2o_n --> coa_n + h_n + hcit_n 14856

HACONTam: hcit_m <=> h2o_m + hacon_C_m 14600

HACONTbm: h2o_m + hacon_C_m <=> hicit_m 16016

HICITDm: hicit_m + nad_m <=> h_m + nadh_m + oxag_m 10428

3HKYNAKGAT: akg_c + hLkynr_c --> 42A3HP24DB_c + glu__L_c 14908 or 15839 or 8540
AATA: 2oxoadp_c + glu__L_c <=> L2aadp_c + akg_c 14908 or 15839
KYNAKGAT: Lkynr_c + akg_c --> 4aphdob_c + glu__L_c 14908 or 15839 or 8540
PHETA1: akg_c + phe__L_c <=> glu__L_c + phpyr_c 14908 or 15839
TRPTA: akg_c + trp__L_c <=> glu__L_c + indpyr_c 12407 or 14908 or 15839
TYRTA: akg_c + tyr__L_c <=> 34hpp_c + glu__L_c 14908 or 15839 or 16065

3HKYNAKGAT: akg_c + hLkynr_c --> 42A3HP24DB_c + glu__L_c 14908 or 15839 or 8540
AATA: 2oxoadp_c + glu__L_c <=> L2aadp_c + akg_c 14908 or 15839
KYNAKGAT: Lkynr_c + akg_c --> 4aphdob_c + glu__L_c 14908 or 15839 or 8540
PHETA1: akg_c + phe__L_c <=> glu__L_c + phpyr_c 14908 or 15839
TRPTA: akg_c + trp__L_c <=> glu__L_c + indpyr_c 12407 or 14908 or 15839
TYRTA: ak

Lysine biosynthesis and aconitase/homoaconitase  
14856 LYS20 is nucl in S. cer, but 14856 has weak sigalP and KEGG matches are mito, add mito reaction  
https://onlinelibrary.wiley.com/doi/pdf/10.1111/mmi.12076  
HICITDm is incorrect, update the reaction to  
hicit_m + nad_m <=> 2oxoadp_m + co2_m + nadh_m  
Change AASAD1 genes to 10220 and 9495, remove AASAD2 (NADH)  
15814 is ALDH7A1 mito degradation, change AASAD3 to mito and remove gene from other rxns  
16833 LYS9 cyto lysine biosynthesis, 10577 AASS mito lysine degradation bifunctional  
Change SACCD1 gene to 16833, SACCD3m and SACCD4m to 10577  
16833 has both SPE3 and LYS9 domains  
Change SPMS and SPRMS genes to 16833

In [106]:
r = sce.reactions.get_by_id('HCITSm').copy()
r.gene_reaction_rule = '14856'
model.remove_reactions(['HCITSn'],remove_orphans=True)
model.add_reactions([r])
model.reactions.get_by_id('HICITDm').add_metabolites({'h_m': -1.0, 'oxag_m': -1.0,
                                                      'co2_m': 1.0, '2oxoadp_m': 1.0})

model.reactions.get_by_id('AASAD1').gene_reaction_rule = '10220 and 9495'
r = hsa.reactions.get_by_id('AASAD3m').copy()
r.gene_reaction_rule = '15814'
model.add_reactions([r])
model.remove_reactions(['AASAD2','AASAD3'],remove_orphans=True)

model.reactions.get_by_id('5HOXINDACTOX').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('ALDD20x').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('ALDD2x').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('BAMPPALDOX').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('GCALDD').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('IMACTD').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('LCADi').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('LCADi_D').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('NABTNO').gene_reaction_rule = '12042 or 13426 or 16323'
model.reactions.get_by_id('PYLALDOX').gene_reaction_rule = '12042 or 13426 or 16323'

model.reactions.get_by_id('SACCD1').gene_reaction_rule = '16833'
model.reactions.get_by_id('SACCD3m').gene_reaction_rule = '10577'
model.reactions.get_by_id('SACCD4m').gene_reaction_rule = '10577'
model.reactions.get_by_id('SPMS').gene_reaction_rule = '16833'
model.reactions.get_by_id('SPRMS').gene_reaction_rule = '16833'

In [107]:
sol = model.optimize()
print(sol.objective_value)

for k, v in sol.shadow_prices.items():
    if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
        print(k, v)

0.0
pe_RT_r 124.47099825740602
mannan_r -1.2447099825740602


In [108]:
with model:
    model.add_reactions([hsa2.reactions.Htr.copy()])
    model.add_reactions([hsa2.reactions.PIter.copy()])
    model.add_reactions([hsa2.reactions.PPItr.copy()])
    model.add_reactions([hsa2.reactions.COAtr.copy()])
    model.add_reactions([hsa2.reactions.NADPHtru.copy()])
    model.add_reactions([hsa2.reactions.NADPtru.copy()])
    model.add_reactions([hsa2.reactions.r0826.copy()]) # inost_r transport
    model.add_reactions([hsa2.reactions.r1159.copy()]) # cdpchol_r transport
    model.add_reactions([hsa2.reactions.FORtr.copy()]) # for_r transport
    model.add_reactions([hsa2.reactions.AMETr.copy()]) # amet_r transport
    model.add_reactions([hsa2.reactions.AHCYStr.copy()]) # ahcys_r transport

    r = hsa2.reactions.Htr.copy()
    r.id = 'ATPter'
    r.name = 'ATP transporter, endoplasmic reticulum'
    model.add_reactions([r])
    r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'atp_c': -1.0, 'atp_r': 1.0})
    r = hsa2.reactions.Htr.copy()
    r.id = 'ADPter'
    r.name = 'ADP transporter, endoplasmic reticulum'
    model.add_reactions([r])
    r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'adp_r': -1.0, 'adp_c': 1.0})
    r = hsa2.reactions.Htr.copy()
    r.id = 'AMPter'
    r.name = 'AMP transporter, endoplasmic reticulum'
    model.add_reactions([r])
    r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'amp_c': -1.0, 'amp_r': 1.0})
    r = hsa2.reactions.Htr.copy()
    r.id = 'CTPter'
    r.name = 'CTP transporter, endoplasmic reticulum'
    model.add_reactions([r])
    r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'ctp_c': -1.0, 'ctp_r': 1.0})
    r = hsa2.reactions.Htr.copy()
    r.id = 'CDPter'
    r.name = 'CDP transporter, endoplasmic reticulum'
    model.add_reactions([r])
    r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'cdp_r': -1.0, 'cdp_c': 1.0})
    r = hsa2.reactions.Htr.copy()
    r.id = 'CMPter'
    r.name = 'CMP transporter, endoplasmic reticulum'
    model.add_reactions([r])
    r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'cmp_c': -1.0, 'cmp_r': 1.0})
    r = hsa2.reactions.Htr.copy()
    r.id = 'SERter'
    r.name = 'L-serine transporter, endoplasmic reticulum'
    model.add_reactions([r])
    r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'ser__L_c': -1.0, 'ser__L_r': 1.0})

    sol = model.optimize()
    print(sol.objective_value)
    for k, v in sol.shadow_prices.items():
        if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
            print(k, v)

1.4134701823753533
adp_c -0.6360615820689088
atp_c -0.6360615820689088
amp_c -0.6360615820689088
cmp_c -0.18846269098338064
glu__L_c -0.14134701823753534
gmp_c -0.6360615820689088
gly_c -1.3877787807814457e-17
ser__L_c -0.259136200102148
cys__L_c -0.2591362001021481
ala__L_c -0.047115672745844966
ump_c -0.18846269098338064
gln__L_c -0.14134701823753534
dgmp_c -0.6360615820689088
pe_RT_r -207.2382865726006
his__L_c -0.37692538196676084
leu__L_c -0.21202052735630303
dtmp_c -0.44759889108552864
trp__L_c -0.47115672745845094
met__L_c -0.3298097092209158
pro__L_c -0.14134701823753534
phe__L_c -0.23557836372922558
tyr__L_c -0.23557836372922558
asp__L_c -0.07067350911876767
ile__L_c -0.14134701823753534
dcmp_c -0.18846269098338064
mannan_r -0.14134701823753537
damp_c -0.6360615820689088
arg__L_c -0.14134701823753534
asn__L_c -0.07067350911876769
glycogen_c -0.14134701823753543
13BDglcn_c -0.14134701823753543
thr__L_c -0.07067350911876769
lys__L_c -0.16490485461045792
val__L_c -0.1413470182375

In [109]:
temp = ['Htr','PIter','PPItr','COAtr','NADPHtru','NADPtru','r0826','r1159','FORtr','AMETr','AHCYStr']
for x in temp:
    model.add_reactions([hsa2.reactions.get_by_id(x).copy()])    

model.reactions.get_by_id('PIter').gene_reaction_rule = ''
model.reactions.get_by_id('r0826').id = 'INOSTter'
model.reactions.get_by_id('r1159').id = 'CDPCHOLter'

r = hsa2.reactions.Htr.copy()
r.id = 'ATPter'
r.name = 'ATP transporter, endoplasmic reticulum'
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'atp_c': -1.0, 'atp_r': 1.0})
r = hsa2.reactions.Htr.copy()
r.id = 'ADPter'
r.name = 'ADP transporter, endoplasmic reticulum'
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'adp_r': -1.0, 'adp_c': 1.0})
r = hsa2.reactions.Htr.copy()
r.id = 'AMPter'
r.name = 'AMP transporter, endoplasmic reticulum'
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'amp_c': -1.0, 'amp_r': 1.0})
r = hsa2.reactions.Htr.copy()
r.id = 'CTPter'
r.name = 'CTP transporter, endoplasmic reticulum'
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'ctp_c': -1.0, 'ctp_r': 1.0})
r = hsa2.reactions.Htr.copy()
r.id = 'CDPter'
r.name = 'CDP transporter, endoplasmic reticulum'
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'cdp_r': -1.0, 'cdp_c': 1.0})
r = hsa2.reactions.Htr.copy()
r.id = 'CMPter'
r.name = 'CMP transporter, endoplasmic reticulum'
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'cmp_c': -1.0, 'cmp_r': 1.0})
r = hsa2.reactions.Htr.copy()
r.id = 'SERter'
r.name = 'L-serine transporter, endoplasmic reticulum'
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_r': -1.0, 'ser__L_c': -1.0, 'ser__L_r': 1.0})

In [110]:
sol = model.optimize()
print(sol.objective_value)

for k, v in sol.shadow_prices.items():
    if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
        print(k, v)

1.4134701823753493
adp_c -0.6360615820689072
atp_c -0.6360615820689072
amp_c -0.6360615820689072
cmp_c -0.18846269098337992
glu__L_c -0.1413470182375349
gmp_c -0.6360615820689072
ser__L_c -0.25913620010214733
cys__L_c -0.2591362001021471
ala__L_c -0.047115672745844994
ump_c -0.18846269098337992
gln__L_c -0.1413470182375349
dgmp_c -0.6360615820689072
pe_RT_r -207.2382865725995
his__L_c -0.37692538196675984
leu__L_c -0.2120205273563024
dtmp_c -0.44759889108552725
trp__L_c -0.4711567274584497
met__L_c -0.32980970922091457
pro__L_c -0.1413470182375349
phe__L_c -0.23557836372922486
tyr__L_c -0.23557836372922486
asp__L_c -0.07067350911876746
ile__L_c -0.14134701823753493
dcmp_c -0.18846269098337992
mannan_r -0.14134701823753482
damp_c -0.6360615820689072
arg__L_c -0.1413470182375349
asn__L_c -0.07067350911876746
glycogen_c -0.14134701823753487
13BDglcn_c -0.14134701823753487
thr__L_c -0.07067350911876746
lys__L_c -0.1649048546104574
val__L_c -0.14134701823753493
ergst_r -0.25913620010213934


In [111]:
# Identify the minimum set of reactions to keep the maximum growth under a reasonable value
remove = ['NA1Htm','UREAtm_1','PYDXO','ASPGLUm','AKGCITtm','AKGMALtm','AKGICITtm','OAAAKGtm']

with model:
    model.remove_reactions(remove, remove_orphans=True)
    sol = model.optimize()
    print(sol.objective_value)

0.12105280269848512


In [112]:
for x in remove:
    with model:
        model.remove_reactions([y for y in remove if not y == x], remove_orphans=True)
        sol = model.optimize()
        print(x, sol.objective_value)

NA1Htm 0.6474314774710036
UREAtm_1 0.12105280269848379
PYDXO 1.3848778722367265
ASPGLUm 1.3733526777687943
AKGCITtm 1.3733526777687748
AKGMALtm 1.3733526777687723
AKGICITtm 1.3733526777687624
OAAAKGtm 1.3733526777687675


In [113]:
for x in remove:
    r = model.reactions.get_by_id(x)
    print(r, r.gene_reaction_rule, ['sce' if r in sce.reactions else 'none'], ['hsa' if r in hsa.reactions else 'none'])

NA1Htm: h_m + na1_c <=> h_c + na1_m 10469 ['none'] ['none']
UREAtm_1: na1_c + urea_c <=> na1_m + urea_m 15807 ['none'] ['none']
PYDXO: 2.0 h2o_c + nh4_c + 0.5 o2_c + pydx_c <=> 2.0 h2o2_c + pydam_c 10680 ['sce'] ['none']
ASPGLUm: asp__L_m + glu__L_c + h_c <=> asp__L_c + glu__L_m + h_m 16799 ['none'] ['hsa']
AKGCITtm: akg_c + cit_m <=> akg_m + cit_c 11740 or 13510 ['none'] ['none']
AKGMALtm: akg_m + mal__L_c <=> akg_c + mal__L_m 11740 or 13510 ['none'] ['hsa']
AKGICITtm: akg_c + icit_m <=> akg_m + icit_c 11740 or 13510 ['none'] ['none']
OAAAKGtm: akg_m + oaa_c <=> akg_c + oaa_m 11740 or 13510 ['none'] ['none']


In [114]:
# first half
temp = ['10469','15807','10680']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10469,"K12041: SLC9A6_7, NHE6_7; solute carrier famil...",A,NHX1,"SLC9A6,SLC9A7,SLC9A9,SLC9A8",NHX1,SLC9A,Not Essential,"plas 24, E.R. 2",GRR*
15807,K20989: DUR3; urea-proton symporter,A,DUR3,,DUR3,,Not Essential,"plas 21, vacu 3, mito 2",GKK*
10680,"K00275: pdxH, PNPO; pyridoxamine 5'-phosphate ...",,PDX3,PNPO,PDX3,PNPO,Not Essential,"cyto_mito 10.166, cyto_nucl 9.833, mito 9.5, c...",LSP*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
10469,0.0479654,0.248788,0.236235,0.0289028,0.231316,0.138566,0.848711,0.978928,0.566191,0.343846,-0.150579
15807,-0.179974,-0.0305221,0.0277623,-0.088636,-0.128047,-0.194151,0.112187,0.0646585,0.0041067,-0.117739,-0.0236412


In [115]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

Kt3g: h_g + k_c <=> h_c + k_g 10469
NA1Htm: h_m + na1_c <=> h_c + na1_m 10469
NAt3_1g: h_g + na1_c <=> h_c + na1_g 10469

UREA2t2: 2.0 h_e + urea_e <=> 2.0 h_c + urea_c 15807
UREAt_1: na1_e + urea_e <=> na1_c + urea_c 15807
UREAtm_1: na1_c + urea_c <=> na1_m + urea_m 15807

PDX5POi: o2_c + pdx5p_c --> h2o2_c + pydx5p_c 10680 or 8901
PYAM5PO: h2o_c + o2_c + pyam5p_c --> h2o2_c + nh4_c + pydx5p_c 10680 or 8901
PYDXNO: o2_c + pydxn_c <=> h2o2_c + pydx_c 10680 or 8901
PYDXO: 2.0 h2o_c + nh4_c + 0.5 o2_c + pydx_c <=> 2.0 h2o2_c + pydam_c 10680
PYDXO_1: h2o_c + o2_c + pydam_c <=> h2o2_c + nh4_c + pydx_c 10680 or 8901



In [116]:
model.remove_reactions(['NA1Htm'],remove_orphans=True)

In [117]:
for r in sorted(model.metabolites.get_by_id('urea_c').reactions, key=lambda x: x.id):
    if len(r.compartments) > 0:
        print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('urea_m').reactions, key=lambda x: x.id):
    if len(r.compartments) > 0:
        print(r, r.gene_reaction_rule)

AGMT: agm_c + h2o_c --> ptrc_c + urea_c 10140
ALLTAHr: alltt_c + h2o_c <=> urdglyc_c + urea_c 14724
ARGN: arg__L_c + h2o_c --> orn_c + urea_c 16181
GUDBUTNAH: 4gudbutn_c + h2o_c --> 4abut_c + urea_c 16181
UREA2t2: 2.0 h_e + urea_e <=> 2.0 h_c + urea_c 15807
UREASE: atp_c + hco3_c + urea_c <=> adp_c + allphn_c + h_c + pi_c 9326
UREAt: urea_e <=> urea_c 13100 or 15470
UREAt_1: na1_e + urea_e <=> na1_c + urea_c 15807
UREAtm: urea_c <=> urea_m 15470
UREAtm_1: na1_c + urea_c <=> na1_m + urea_m 15807

AGMTm: agm_m + h2o_m --> ptrc_m + urea_m 10140
ARGNm: arg__L_m + h2o_m --> orn_m + urea_m 16181
UREAtm: urea_c <=> urea_m 15470
UREAtm_1: na1_c + urea_c <=> na1_m + urea_m 15807


In [118]:
temp = ['15807','10140','15470','16181']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
15807,K20989: DUR3; urea-proton symporter,A,DUR3,,DUR3,,Not Essential,"plas 21, vacu 3, mito 2",GKK*
10140,K01480: speB; agmatinase,S,,AGMAT,,AGMAT,Not Essential,"extr 23, mito 2, plas 2",VRV*
15470,K03441: GLP-F; aquaglyceroporin related protei...,,"YFL054C,FPS1","AQP3,AQP9",YFL054C,AQP3,Not Essential,"plas 17, nucl 6, mito 1, cyto 1, E.R. 1, vacu ...",WRQ*
16181,"K01476: E3.5.3.1, rocF, arg; arginase",,CAR1,"ARG1,ARG2",CAR1,ARG1,Not Essential,"cyto 25.5, cyto_nucl 14",TLL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
15807,-0.179974,-0.0305221,0.0277623,-0.088636,-0.128047,-0.194151,0.112187,0.0646585,0.0041067,-0.117739,-0.0236412
10140,0.0870947,-0.0379351,0.121567,0.232117,0.32541,-0.1721,-0.266461,-0.274505,0.39974,0.0193737,0.0688812
15470,0.00318651,0.0512761,-0.167947,0.0916783,0.0498898,-0.185868,0.158407,-0.399259,0.128998,-0.140828,-0.115934


In [119]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

UREA2t2: 2.0 h_e + urea_e <=> 2.0 h_c + urea_c 15807
UREAt_1: na1_e + urea_e <=> na1_c + urea_c 15807
UREAtm_1: na1_c + urea_c <=> na1_m + urea_m 15807

AGMT: agm_c + h2o_c --> ptrc_c + urea_c 10140
AGMTm: agm_m + h2o_m --> ptrc_m + urea_m 10140

GLYCt: glyc_c <=> glyc_e 13100 or 15470
L_LACtcm: lac__L_c --> lac__L_m 15470
UREAt: urea_e <=> urea_c 13100 or 15470
UREAtm: urea_c <=> urea_m 15470

ARGN: arg__L_c + h2o_c --> orn_c + urea_c 16181
ARGNm: arg__L_m + h2o_m --> orn_m + urea_m 16181
GUDBUTNAH: 4gudbutn_c + h2o_c --> 4abut_c + urea_c 16181



In [120]:
model.remove_reactions(['UREAtm_1','UREAt_1','ARGNm','AGMT','L_LACtcm','LCADm','LCADi_Dm'],remove_orphans=True)

In [121]:
for r in sorted(model.metabolites.get_by_id('pydx_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('pydam_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

PYDXDH_1: h2o_c + o2_c + pydx_c --> 4pyrdx_c + h2o2_c + h_c 15962
PYDXK: atp_c + pydx_c --> adp_c + h_c + pydx5p_c 14545
PYDXNO: o2_c + pydxn_c <=> h2o2_c + pydx_c 10680 or 8901
PYDXO: 2.0 h2o_c + nh4_c + 0.5 o2_c + pydx_c <=> 2.0 h2o2_c + pydam_c 10680
PYDXOR: h_c + nadph_c + pydx_c <=> nadp_c + pydxn_c 11550 or 14369
PYDXO_1: h2o_c + o2_c + pydam_c <=> h2o2_c + nh4_c + pydx_c 10680 or 8901
PYDXPP: h2o_c + pydx5p_c --> pi_c + pydx_c 13044 or 14545

HYPOE: h2o_c + pyam5p_c --> pi_c + pydam_c 13044
PYDAMK: atp_c + pydam_c --> adp_c + h_c + pyam5p_c 14545
PYDXO: 2.0 h2o_c + nh4_c + 0.5 o2_c + pydx_c <=> 2.0 h2o2_c + pydam_c 10680
PYDXO_1: h2o_c + o2_c + pydam_c <=> h2o2_c + nh4_c + pydx_c 10680 or 8901


In [122]:
temp = ['13044','14545','11550','14369','10680','15962','8901']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
13044,K01101: E3.1.3.41; 4-nitrophenyl phosphatase,,PHO13,"PGP,PDXP",PHO13,PDXP,Not Essential,"cyto 17.5, cyto_nucl 12.5, nucl 4.5, pero 3",LAQ*
14545,"K00868: pdxK, pdxY; pyridoxine kinase",,"BUD16,BUD17",PDXK,BUD16,PDXK,Not Essential,"mito 10.5, cyto_mito 10.333, cyto 9, extr 6, c...",SGV*
11550,"KOG1575: Voltage-gated shaker-like K+ channel,...",A,,,YPR127W,AKR7L,Not Essential,"mito 8, cyto 7.5, cyto_nucl 7.5, nucl 6.5, pero 4",MDE*
14369,"KOG1575: Voltage-gated shaker-like K+ channel,...",,,,YPR127W,KCNAB,Not Essential,"cyto 23, cysk 4",LDG*
10680,"K00275: pdxH, PNPO; pyridoxamine 5'-phosphate ...",,PDX3,PNPO,PDX3,PNPO,Not Essential,"cyto_mito 10.166, cyto_nucl 9.833, mito 9.5, c...",LSP*
15962,K00106: XDH; xanthine dehydrogenase/oxidase,,,"AOX1,XDH",,XDHe,Not Essential,"cyto 21.5, cyto_nucl 12.5, nucl 2.5",VCI*
8901,"K17759: AIBP, nnrE; NAD(P)H-hydrate epimerase",,YNL200C,NAXE,YNL200C,NAXE,Not Essential,"cyto 12.5, cyto_nucl 9.5, extr 5, mito 4, nucl...",RKE*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
14545,0.0209469,-0.292783,-0.0266595,-0.0118151,-0.199009,0.101022,0.0768783,0.290596,0.207443,0.17306,-0.249223
11550,0.0774105,0.144846,-0.018435,0.121283,0.226393,0.108317,-0.0519291,0.166514,0.195984,0.0572959,0.122298
14369,0.323773,-0.134946,0.179229,0.146106,0.0898245,0.388908,0.140926,0.0150337,-0.223792,0.06217,-0.00291277
15962,0.0348718,-0.0178509,0.114898,0.0600965,0.0873949,0.05488,0.105901,0.153482,0.0853636,0.14613,-0.246039
8901,-0.00988828,0.196981,0.158386,0.096119,-0.0526187,-0.0215802,0.104289,0.000964794,0.377928,0.25981,0.0721251


In [123]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

G3PT: glyc3p_c + h2o_c --> glyc_c + pi_c 13044 or 13413
HYPOE: h2o_c + pyam5p_c --> pi_c + pydam_c 13044
NTD2: h2o_c + ump_c --> pi_c + uri_c 13044 or 16648 or 9995
NTD4: cmp_c + h2o_c --> cytd_c + pi_c 13044 or 16648 or 9995
NTD7: amp_c + h2o_c --> adn_c + pi_c 13044 or 16648
NTD9: gmp_c + h2o_c --> gsn_c + pi_c 13044 or 16648
PDXPP: h2o_c + pdx5p_c --> pi_c + pydxn_c 13044 or 14545
PYDXPP: h2o_c + pydx5p_c --> pi_c + pydx_c 13044 or 14545
R5PP: h2o_c + r5p_c --> pi_c + rib__D_c 13044 or 14546 or 8576

PDXPP: h2o_c + pdx5p_c --> pi_c + pydxn_c 13044 or 14545
PYDAMK: atp_c + pydam_c --> adp_c + h_c + pyam5p_c 14545
PYDXK: atp_c + pydx_c --> adp_c + h_c + pydx5p_c 14545
PYDXNK: atp_c + pydxn_c --> adp_c + h_c + pdx5p_c 14545
PYDXPP: h2o_c + pydx5p_c --> pi_c + pydx_c 13044 or 14545

ALR2x: h_c + mthgxl_c + nadh_c --> acetol_c + nad_c 11550 or 14369
PYDXOR: h_c + nadph_c + pydx_c <=> nadp_c + pydxn_c 11550 or 14369

ALR2x: h_c + mthgxl_c + nadh_c --> acetol_c + nad_c 11550 or 14369
PYDXO

In [124]:
temp = ['13413','16648','12939','9995','11513','14546','8576']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
13413,K01111: E3.1.3.68; 2-deoxyglucose-6-phosphatase,,"GPP2,GPP1,DOG2,DOG1",,DOG1,,Not Essential,"cysk 17, cyto 10",GTA*
16648,KOG2157: Predicted tubulin-tyrosine ligase,,,,PBY1,,Not Essential,"mito 17, cyto 5, nucl 4",WKI*
12939,KOG4419: 5' nucleotidase,,YHR202W,,YHR202W,,Not Essential,"cyto 12.5, cyto_nucl 12, nucl 10.5, pero 3",EWN*
9995,K18551: SDT1; pyrimidine and pyridine-specific...,,"SDT1,PHM8",,SDT1,,Not Essential,"mito 17, nucl 5, pero 3",GSS*
11513,HMMPfam:haloacid dehalogenase-like hydrolase:P...,,,,,,Not Essential,"mito 10, cyto 7.5, cyto_nucl 6.5, pero 5, nucl...",AYP*
14546,K17623: HDHD1; pseudouridine 5'-phosphatase,,YKL033W-A,PUDP,YKL033W-A,PUDP,Not Essential,"cyto 10.5, cysk 9, cyto_nucl 8, nucl 4.5",YDQ*
8576,"K19270: yfbT, yniC; sugar-phosphatase",,,,GPP2,,Not Essential,"cyto 18, cysk 8",FVV*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
13413,0.0653619,-0.121519,0.17951,0.0909367,0.399522,0.263557,-0.530393,0.667075,0.651914,0.610799,0.728724
12939,-0.0629916,0.0602598,0.0552619,0.0636762,0.383564,-0.0264626,-0.0932594,0.18259,0.46209,0.847028,0.508303
9995,0.368148,0.35758,0.0592439,0.409551,0.167506,0.120755,0.814853,-0.430807,-1.35726,-0.190163,1.44696
11513,0.0450204,0.0302811,-0.04327,0.0507433,-0.535173,0.0529926,-0.0692324,-0.128559,-0.175321,-0.272272,-0.0418747
14546,0.441895,0.052392,0.302713,0.220549,-0.147649,0.167133,-0.101398,-0.225625,-0.193782,-0.238061,0.105055
8576,0.0928334,0.0403074,0.342139,0.0272539,1.10922,0.219086,-0.299253,0.0333091,0.136365,-0.0567877,0.190766


In [125]:
model.reactions.get_by_id('G3PT').gene_reaction_rule = '13413'
model.reactions.get_by_id('PDXPP').gene_reaction_rule = '13044'
model.reactions.get_by_id('PYDXPP').gene_reaction_rule = '13044'
model.reactions.get_by_id('R5PP').gene_reaction_rule = '14546 or 8576'
# 16648 is 5'-nucleotidase SurE
model.reactions.get_by_id('NTD2').gene_reaction_rule = '16648 or 9995'
model.reactions.get_by_id('NTD4').gene_reaction_rule = '16648 or 9995'
model.reactions.get_by_id('NTD7').gene_reaction_rule = '16648'
model.reactions.get_by_id('NTD9').gene_reaction_rule = '16648'
# 8901 is NAXE, remove it from pyridoxine reactions
model.reactions.get_by_id('PDX5POi').gene_reaction_rule = '10680'
model.reactions.get_by_id('PYAM5PO').gene_reaction_rule = '10680'
model.reactions.get_by_id('PYDXNO').gene_reaction_rule = '10680'
model.reactions.get_by_id('PYDXO_1').gene_reaction_rule = '10680'
# 11550 or 14369 YPR127W pyridoxine 4-dehydrogenase, remove ALR2x (E. coli ydjG)
# 15962 is XDH, but not AOX1, remove AOX1 reactions
model.remove_reactions(['ALR2x','PYDXO','5HOXINDACTO2OX','PYDXDH_1'],remove_orphans=True)

In [126]:
# second half
# https://www.sciencedirect.com/science/article/pii/S0005272806001630#fig3
# https://www.sciencedirect.com/science/article/pii/S0098299712000519#f0005
# https://www.sciencedirect.com/science/article/pii/S016748891630060X#f0015
temp = ['16799','11740','9011','13510','10514','16746','10635','12605','15874','8889']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
16799,"K15105: SLC25A12_13, AGC; solute carrier famil...",S,AGC1,"SLC25A12,SLC25A13",AGC1,SLC25,Not Essential,"mito 22, cyto 3",SQK*
11740,"K15103: UCP2_3, SLC25A8_9; solute carrier fami...",,,UCP2,OAC1,SLC25,Not Essential,"mito 15, cysk 6, cyto 4.5, cyto_nucl 3",PPV*
9011,"K15117: SLC25A34_35, OAC1; solute carrier fami...",S,OAC1,SLC25A35,OAC1,SLC25,Not Essential,"extr 14, mito 11",ARD*
13510,"K13577: SLC25A10, DIC; solute carrier family 2...",,DIC1,"SLC25A11,RP13-1032I1.10",DIC1,SLC25,Not Essential,"mito 15, extr 7, cyto 3",GTP*
10514,"K15100: SLC25A1, CTP; solute carrier family 25...",S,CTP1,SLC25A1,CTP1,SLC25A,Not Essential,"extr 13, mito 5, E.R. 3, vacu 3, cyto_mito 3",RAV*
16746,KOG0756: Mitochondrial tricarboxylate/dicarbox...,,YHM2,,YHM2,,Not Essential,"cyto 13, mito 10, cysk 2",NKK*
10635,"K15110: SLC25A21, ODC; solute carrier family 2...",,"ODC2,ODC1",SLC25A21,ODC2,SLC25,Not Essential,"mito 13, cyto 10, extr 3",PYI*
12605,"K15100: SLC25A1, CTP; solute carrier family 25...",S,SFC1,,SFC1,SLC25A,Not Essential,"mito 11, cyto 9.5, cyto_nucl 5.5, plas 3",YSE*
15874,"K15102: SLC25A3, PHC, PIC; solute carrier fami...",,"MIR1,PIC2",SLC25A3,MIR1,SLC25,Not Essential,"mito 12, extr 7, cyto 5.5, cyto_nucl 3.5",HKD*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16799,0.00860844,0.284126,-0.110846,-0.472156,-0.0178013,-0.848331,-1.27882,-1.48965,-0.207239,-0.0405575,0.0403966
11740,0.109553,0.00205556,-0.00502673,-0.149136,0.237068,0.0193826,0.313053,0.0534518,0.095281,0.428816,0.101248
9011,0.0241941,-0.272725,0.161747,-0.269852,0.0798061,-0.031333,-0.0244879,-0.492747,0.287088,0.0815357,-0.107048
13510,0.206135,0.686782,0.453603,-0.0292148,-0.421161,-0.352184,0.219954,-0.116016,-0.17613,0.0633609,0.114048
10514,0.278631,-0.163472,-0.438358,-0.0852117,0.465255,-0.211769,0.0577349,-1.659,-0.743333,-0.169441,0.0921063
10635,0.205165,0.0964621,0.203563,0.261393,-3.43862,-4.08137,-0.480008,-0.819799,0.253622,0.413434,-0.0938174
12605,-0.250531,-1.12124,-1.27614,-1.68184,-1.69155,-1.56883,0.318931,0.287611,1.11429,0.16432,2.2361
15874,-0.268098,-0.0197262,0.0222415,-0.262453,0.268151,-0.511158,0.130008,0.406326,0.236206,-0.0925991,0.264876


In [127]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

3SALAASPm: 3sala_m + asp__L_c <=> 3sala_c + asp__L_m 16799
ASPGLU2m: asp__L_m + glu__L_c --> asp__L_c + glu__L_m 16799
ASPGLUm: asp__L_m + glu__L_c + h_c <=> asp__L_c + glu__L_m + h_m 16799
GLUt7m: glu__L_c --> glu__L_m 16799

AKGCITtm: akg_c + cit_m <=> akg_m + cit_c 11740 or 13510
AKGICITtm: akg_c + icit_m <=> akg_m + icit_c 11740 or 13510
AKGMALtm: akg_m + mal__L_c <=> akg_c + mal__L_m 11740 or 13510
CITtam: cit_c + mal__L_m <=> cit_m + mal__L_c 10514 or 11740 or 13510
CITtcm: cit_c + icit_m <=> cit_m + icit_c 10514 or 11740 or 13510
Htm: h_c --> h_m 11740
MALICITtm: icit_m + mal__L_c <=> icit_c + mal__L_m 11740 or 13510
MALOAAtm: mal__L_m + oaa_c <=> mal__L_c + oaa_m 11740 or 13510
MALtm: mal__L_c + pi_m <=> mal__L_m + pi_c 11740 or 13510
OAAAKGtm: akg_m + oaa_c <=> akg_c + oaa_m 11740 or 13510
OAACITtm: cit_m + oaa_c <=> cit_c + oaa_m 11740 or 13510
OAAICITtm: icit_m + oaa_c <=> icit_c + oaa_m 11740 or 13510
SUCCtm: pi_m + succ_c --> pi_c + succ_m 11740 or 13510
SUCFUMtm: fum_m + 

In [128]:
# 16799 AGC1, also acts as glutamate uniporter, ASPGLU2m incorrect
# https://onlinelibrary.wiley.com/doi/full/10.1046/j.1365-2958.2003.03742.x
# ASPGLUm asp__L_m + glu__L_c + h_c -> asp__L_c + glu__L_m + h_m correct
model.reactions.get_by_id('ASPGLUm').lower_bound = 0.0
model.remove_reactions(['ASPGLU2m'], remove_orphans=True)
# 11740 UCP2, asp/oaa/mal export in exchange with h + pi remove 11740 from other reactions
# https://www.pnas.org/content/111/3/960
# Htm is by UCP1, but no UCP1 is found in Rhodo
r = model.reactions.get_by_id('OAAt2m').copy()
r.id = 'UCP2ASPtm'
r.name = 'Uncoupling protein 2 (aspartate), mitochondrial'
r.gene_reaction_rule = '11740'
r.lower_bound = 0.0
model.add_reactions([r])
r.add_metabolites({'oaa_c': 1.0, 'oaa_m': -1.0, 'pi_c': -1.0, 'pi_m': 1.0,
                   'asp__L_m': -1.0, 'asp__L_c': 1.0})
r = model.reactions.get_by_id('OAAt2m').copy()
r.id = 'UCP2MALtm'
r.name = 'Uncoupling protein 2 (malate), mitochondrial'
r.gene_reaction_rule = '11740'
r.lower_bound = 0.0
model.add_reactions([r])
r.add_metabolites({'oaa_c': 1.0, 'oaa_m': -1.0, 'pi_c': -1.0, 'pi_m': 1.0,
                   'mal__L_m': -1.0, 'mal__L_c': 1.0})
r = model.reactions.get_by_id('OAAt2m').copy()
r.id = 'UCP2OAAtm'
r.name = 'Uncoupling protein 2 (oxaloacetate), mitochondrial'
r.gene_reaction_rule = '11740'
r.lower_bound = 0.0
model.add_reactions([r])
r.add_metabolites({'oaa_c': 2.0, 'oaa_m': -2.0, 'pi_c': -1.0, 'pi_m': 1.0})
model.remove_reactions(['Htm','OAAt2m'], remove_orphans=True)
# 9011 OAC1 transports oaa, so4, thiosulfate, 3c3hmp_m (a-isopropylmalate)
# http://www.jbc.org/content/274/32/22184.full.pdf
# http://www.jbc.org/content/283/42/28445.full
# add 3c3hmp_m export / oaa import
r = model.reactions.get_by_id('OAAAKGtm').copy()
r.id = 'OAAIPMtm'
r.name = 'Dicarboxylate/tricarboxylate carrier (oaa:3c3hmp), mitochondrial'
r.gene_reaction_rule = '9011'
r.lower_bound = 0.0
model.add_reactions([r])
r.add_metabolites({'akg_m': 1.0, 'akg_c': -1.0, '3c3hmp_m': -1.0, '3c3hmp_c': 1.0})
model.remove_reactions(['OAAAKGtm','OAACITtm','OAAICITtm'], remove_orphans=True)
# 13510 DIC1 imports malate and succinate in exchange of pi, so4, thiosulfate
model.reactions.get_by_id('MALtm').gene_reaction_rule = '13510'
model.reactions.get_by_id('SUCCtm').gene_reaction_rule = '13510'
model.reactions.get_by_id('MALtm').lower_bound = 0.0
model.reactions.get_by_id('SUCCtm').lower_bound = 0.0
model.remove_reactions(['MALOAAtm','MALSO3tm','MALSO4tm','MALTSULtm'], remove_orphans=True)
# 10514 CTP1 citrate exchange with icit / mal
model.reactions.get_by_id('CITtam').gene_reaction_rule = '10514'
model.reactions.get_by_id('CITtcm').gene_reaction_rule = '10514'
model.remove_reactions(['CITtbm','MALICITtm'], remove_orphans=True)
# 16746 YHM2 exports citrate and imports akg into mito
model.reactions.get_by_id('AKGCITtm').gene_reaction_rule = '16746'
model.reactions.get_by_id('AKGCITtm').lower_bound = 0.0
model.remove_reactions(['AKGICITtm'], remove_orphans=True)
# 10635 ODC1/ODC2 exports akg/2oxoadp_c in exchange with mal
model.reactions.get_by_id('AKGMALtm').gene_reaction_rule = '10635'
model.reactions.get_by_id('AKGMALtm').lower_bound = 0.0
model.remove_reactions(['2AMADPTm','OXO2Ctm'], remove_orphans=True)
# 12605 SFC1 imports succ and exports fum
model.reactions.get_by_id('SUCFUMtm').gene_reaction_rule = '12605'
model.reactions.get_by_id('SUCFUMtm').lower_bound = 0.0
model.remove_reactions(['FUMSO3tm','FUMSO4tm','FUMTSULtm','FUMtm'], remove_orphans=True)
# 15874 MIR1 or 8889 PIC2 imports pi
model.reactions.get_by_id('PIt2m').lower_bound = 0.0

In [129]:
temp = list(Annotation.index[Annotation['Combined Annotations'].str.contains('mitochondrial') &
                            Annotation['Combined Annotations'].str.contains('carrier')]) + \
       list(Annotation.index[Annotation['Combined Annotations'].str.contains('SLC25A')])
temp = set(temp) - set(['16799','11740','9011','13510','10514','16746','10635','12605','15874','8889'])
temp = sorted(list(temp), key=lambda x: Annotation.loc[x, 'Combined Annotations'])
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
12704,"K05863: SLC25A4S, ANT; solute carrier family 2...",,"PET9,AAC3","SLC25A4,SLC25A5,SLC25A6,SLC25A31",PET9,SLC25A,Essential,"mito 17.5, cyto_mito 11.166, extr 5, cyto 3.5",GSG*
16095,"K13354: SLC25A17, PMP34; solute carrier family...",,,SLC25A17,FLX1,SLC25,Not Essential,"mito 11, extr 7, cyto 5, pero 4",AKA*
9695,K14684: SLC25A23S; solute carrier family 25 (m...,,SAL1,"SLC25A23,SLC25A25,SLC25A24",SAL1,SLC25,Not Essential,"mito 10, cyto 9, cyto_nucl 9, nucl 7",EES*
16440,K14684: SLC25A23S; solute carrier family 25 (m...,,YPR011C,,YPR011C,SLC25,Not Essential,"mito 12, cyto 12, cyto_mito 12",SEA*
8916,"K15084: SLC25A16, GDA, LEU5; solute carrier fa...",,LEU5,"SLC25A16,SLC25A42",LEU5,SLC25,Essential,"mito 13, nucl 8, cyto 4",EGR*
12518,"K15101: SLC25A2_15, ORNT; solute carrier famil...",,YMC2,,YMC2,SLC25,Not Essential,"mito 10, cyto 6, mito_nucl 6, extr 4, cyto_nucl 4",TSL*
9331,"K15109: SLC25A20_29, CACT, CACL, CRC1; solute ...",,CRC1,SLC25A20,CRC1,SLC25A,Not Essential,"mito 13.5, cyto_mito 12.5, cyto 10.5",ALF*
10149,"K15109: SLC25A20_29, CACT, CACL, CRC1; solute ...",,,,YMC2,SLC25,Not Essential,"mito 11, plas 10, nucl 1, cyto 1, extr 1, cyto...",TSD*
12032,K15111: SLC25A26; solute carrier family 25 (mi...,,PET8,SLC25A26,PET8,SLC25,Not Essential,"mito 10, extr 8, cyto 7",ERD*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16095,-0.0218641,-0.177959,0.247816,0.167411,-0.322285,-0.0863019,-0.350025,0.0493235,0.345264,0.262751,-0.385954
9695,0.0480373,0.164832,0.123385,-0.136937,0.0797862,-0.114876,-0.0114183,-0.668377,-0.425499,0.0532996,0.309092
16440,-0.0368947,0.0914269,-0.305477,-0.0597754,-0.0287667,-0.128169,-0.563395,0.000584502,0.559049,0.77942,0.603735
12518,0.112526,0.0372373,-0.100235,0.0203073,0.136688,0.0815494,-0.292448,-0.26549,0.0219979,0.091007,0.19193
9331,0.317304,0.660965,-0.231468,-1.77645,0.262022,-1.83089,-3.79725,-6.14964,-0.943653,-0.572548,-1.19392
10149,-0.075474,-0.4456,-0.241651,0.0656419,-0.293397,-0.0232707,-0.142486,0.752866,-0.199261,-0.496042,-0.194102
12032,-0.956871,-0.659087,-0.293917,-1.14194,-0.341075,-0.533584,0.866135,-2.15565,-2.81679,1.70697,-0.147228
13319,-0.424986,-0.449847,-0.398549,-0.266453,-0.490897,0.0684878,0.363013,-0.485849,-1.56767,-0.465167,-0.150008
8434,-0.288681,-0.0976237,-0.263692,-0.498388,-3.67827,-0.708144,-0.206495,0.00227462,-0.0963591,-0.166823,0.35102


In [130]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ATPtm: adp_c + atp_m --> adp_m + atp_c 12704

16095 no reactions

9695 no reactions

16440 no reactions

COAtim: coa_c --> coa_m 8916

12518 no reactions

ACRNtm: acrn_c --> acrn_m 9331
ARACHCRNt: arachcrn_c --> arachcrn_m 9331
C160CRNt: pmtcrn_c --> pmtcrn_m 9331
C161CRN2t: hdd2crn_c --> hdd2crn_m 9331
C161CRNt: hdcecrn_c --> hdcecrn_m 9331
C181CRNt: odecrn_c --> odecrn_m 9331
CLPNDCRNt: clpndcrn_c --> clpndcrn_m 9331
CRNtim: crn_m --> crn_c 9331
DCSPTN1CRNt: dcsptn1crn_c --> dcsptn1crn_m 9331
ELAIDCRNt: elaidcrn_c --> elaidcrn_m 9331
HEXCCRNt: hexccrn_c --> hexccrn_m 9331
HPDCACRNt: hpdcacrn_c --> hpdcacrn_m 9331
LNLNCGCRNt: lnlncgcrn_c --> lnlncgcrn_m 9331
NRVNCCRNt: nrvnccrn_c --> nrvnccrn_m 9331
PCRNtm: pcrn_c <=> pcrn_m 9331
PTDCACRNt: ptdcacrn_c --> ptdcacrn_m 9331
TETTET6CRNt: tettet6crn_c --> tettet6crn_m 9331
TTDCRNt: ttdcrn_c --> ttdcrn_m 9331

ACRNtp: acrn_x --> acrn_c 10149
PCRNtc: pcrn_x --> pcrn_c 10149

AMETt2m: ahcys_m + amet_c <=> ahcys_c + amet_m 12032
AMETtm: amet_c

In [131]:
# 12704 ANT, ok
# 16095 FAD, no rxn
# 9695 ADP/ATP transporter or ATP-Mg/phosphate carriers, no rxn
# 16440 YPR011C APS and PAPS (thought in exchange for pi or so4), no rxn
# 8916 CoA, ok
# 12518 SLC25A2_15, ORNT ornithine carrier or YMC2?, unclear, no rxn
# 13319 ORT1 exports ornithine in exchange of proton, can import arg or lys in exchange with orn
model.reactions.get_by_id('ORNt3m').lower_bound = 0.0
r = model.reactions.get_by_id('ORNt3m').copy()
r.id = 'ORNARGtm'
r.lower_bound = 0.0
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_m': -1.0, 'arg__L_c': -1.0, 'arg__L_m': 1.0})
r = model.reactions.get_by_id('ORNt3m').copy()
r.id = 'ORNLYStm'
r.lower_bound = 0.0
model.add_reactions([r])
r.add_metabolites({'h_c': 1.0, 'h_m': -1.0, 'lys__L_c': -1.0, 'lys__L_m': 1.0})
# 10149 basic amino acids with a preference for arg and lys, less ornithine and histidine
r = model.reactions.get_by_id('GLUt7m').copy()
r.id = 'ARGt7m'
r.lower_bound = 0.0
model.add_reactions([r])
r.add_metabolites({'glu__L_c': 1.0, 'glu__L_m': -1.0, 'arg__L_c': -1.0, 'arg__L_m': 1.0})
r = model.reactions.get_by_id('GLUt7m').copy()
r.id = 'LYSt7m'
r.lower_bound = 0.0
model.add_reactions([r])
r.add_metabolites({'glu__L_c': 1.0, 'glu__L_m': -1.0, 'lys__L_c': -1.0, 'lys__L_m': 1.0})
# 9331 carnitine, ok
# 10232 PET8 (SAM5) mito amet import by uniport or exchange with ahcys, ok
# 12446 MRS3/MRS4 mito iron transport, no rxn
r = hsa.reactions.get_by_id('FE2tm').copy()
r.gene_reaction_rule = '12446'
model.add_reactions([r])
# 14159 FLX1 FAD, ok
# 10961 YIA6 mito NAD import, no rxn
# 12387 RIM2 pyrimidine nucleotide, proton coupled, ok
# RIM2 also co-transports fe2 in MRS3 and MRS4 mutants
# http://www.biochemj.org/content/455/1/57.long
r1 = model.reactions.get_by_id('CTPtm').copy()
r1.id = 'CTPtm2'
r2 = model.reactions.get_by_id('UTPtm').copy()
r2.id = 'UTPtm2'
model.add_reactions([r1,r2])
model.reactions.get_by_id('CTPtm2').add_metabolites({'h_c': 2.0, 'h_m': -2.0, 'fe2_c': -1.0, 'fe2_m': 1.0})
model.reactions.get_by_id('UTPtm2').add_metabolites({'h_c': 2.0, 'h_m': -2.0, 'fe2_c': -1.0, 'fe2_m': 1.0})
model.remove_reactions(['CMPtm'], remove_orphans=True)
# 8434 HEM25 mito glycine
# https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1005783
# mito glycine needed for heme biosynthesis
# http://www.jbc.org/content/291/38/19746/F10.expansion.html
r = hsa.reactions.get_by_id('GLYtm').copy()
r.gene_reaction_rule = '8434'
model.add_reactions([r])
# 11375 MTM1 high affinity pyridoxal 5'-phosphate transporter
r = hsa.reactions.get_by_id('PYDX5Ptm').copy()
r.gene_reaction_rule = '11375'
model.add_reactions([r])
# 16145 ANT1 perox ATP transporter is electroneutral, should bring h from cyto to pero
model.reactions.get_by_id('ATP2tp_H').add_metabolites({'h_c': -3.0, 'h_x': 3.0})

siroheme biosynthesis  
ALASm: gly_m + h_m + succoa_m --> 5aop_m + co2_m + coa_m 8677  
Add 5AOPtm: 5aop_c <=> 5aop_m  
PPBNGS: 2.0 5aop_c --> 2.0 h2o_c + h_c + ppbng_c 13394  
HMBS: h2o_c + 4.0 ppbng_c --> hmbil_c + 4.0 nh4_c 14718  
UPP3S: hmbil_c --> h2o_c + uppg3_c 10515  
UPP3MT: 2.0 amet_c + uppg3_c --> 2.0 ahcys_c + dscl_c + h_c 11050  
SHCHD2: dscl_c + nad_c --> h_c + nadh_c + scl_c 11050 or 13569  
SHCHD: dscl_c + nadp_c --> h_c + nadph_c + scl_c 13569  
SHCHF: fe2_c + scl_c --> 3.0 h_c + sheme_c 11050 or 13569  

protoheme biosynthesis  
UPPDC1: 4.0 h_c + uppg3_c --> 4.0 co2_c + cpppg3_c 14179  
CPPPGO: cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c 14197  
Add pppg9 mito transport, PPPG9tm  
PPPGOm: 3.0 o2_m + 2.0 pppg9_m --> 6.0 h2o_m + 2.0 ppp9_m 11867  
Add FCLTm: fe2_m + ppp9_m --> 2.0 h_m + pheme_m 16275  
16275 is two genes in one gene model (ATPase + HEM15) -> need to be separared  
HEMEOSm: frdp_m + h2o_m + pheme_m --> hemeO_m + ppi_m 11103  
HEMEASm: 2.0 h_m + hemeO_m + 2.0 nadph_m + 2.0 o2_m --> 3.0 h2o_m + hemeA_m + 2.0 nadp_m 10228 and 15585 and 16775  

In [132]:
temp = ['8677','13394','14718','10515','11050','13569','14197','11867','16275','11103','10228','15585','16775']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8677,"K00643: E2.3.1.37, ALAS; 5-aminolevulinate syn...",,HEM1,"ALAS1,ALAS2,GCAT",HEM1,ALAS2,Essential,"mito 20, extr 4, cyto 3",ASA*
13394,"K01698: hemB, ALAD; porphobilinogen synthase",,HEM2,ALAD,HEM2,ALAD,Essential,"mito 12, cyto 7.5, cyto_nucl 5.5, cysk 3, nucl...",DKQ*
14718,"K01749: hemC, HMBS; hydroxymethylbilane synthase",,HEM3,HMBS,HEM3,HMBS,Essential,"mito 18, cyto 6, cyto_nucl 5.5",VGW*
10515,"K01719: hemD, UROS; uroporphyrinogen-III synthase",,HEM4,UROS,HEM4,UROS,Not Essential,"mito 13, mito_nucl 10.666, cyto_mito 10.166, c...",QDE*
11050,K00589: MET1; uroporphyrin-III C-methyltransfe...,S,MET1,,MET1,,Not Essential,"cyto 10, mito 8, cyto_nucl 8",SAE*
13569,K02304: MET8; precorrin-2 dehydrogenase / siro...,,MET8,,MET8,,Not Essential,"mito 23, cyto 4",RRQ*
14197,"K00228: CPOX, hemF; coproporphyrinogen III oxi...",S,HEM13,CPOX,HEM13,CPOX,Not Essential,"plas 7, mito 4, extr 4, nucl 3, E.R. 3, golg 3...",EWA*
11867,"K00231: PPOX, hemY; oxygen-dependent protoporp...",A,HEM14,PPOX,,PPOX,Essential,"mito 22, cyto 3",WEL*
16275,K14951: ATP13A3_4_5; cation-transporting ATPas...,,YPK9,"ATP13A2,ATP13A3,ATP13A4,ATP13A5",YPK9,ATP13,Not Essential,"plas 23, E.R. 2",VSA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11050,-0.248795,0.364452,0.238288,0.242943,0.425463,-1.26448,-0.264639,-1.31736,-2.83255,-0.0264744,-0.0540932
13569,-0.599414,-0.697888,-0.446222,-0.131288,0.0469392,-1.82967,-0.719508,-1.34086,-3.11283,0.959913,-0.789865
14197,0.156461,-0.0582697,0.147453,-0.0716227,-0.190346,-0.0878437,-0.0200181,-0.202957,0.359937,0.2702,-0.0314271
16275,-0.269023,-0.762556,-1.38232,-0.0913242,-0.0296838,0.0389366,-0.116315,-0.526475,-0.689562,-0.642096,-0.0125306


In [133]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ALASm: gly_m + h_m + succoa_m --> 5aop_m + co2_m + coa_m 8677
GLYAT: accoa_c + gly_c <=> 2aobut_c + coa_c 8677
GLYATm: accoa_m + gly_m <=> 2aobut_m + coa_m 8677

PPBNGS: 2.0 5aop_c --> 2.0 h2o_c + h_c + ppbng_c 13394

HMBS: h2o_c + 4.0 ppbng_c --> hmbil_c + 4.0 nh4_c 14718

UPP3S: hmbil_c --> h2o_c + uppg3_c 10515

SHCHD2: dscl_c + nad_c --> h_c + nadh_c + scl_c 11050 or 13569
SHCHF: fe2_c + scl_c --> 3.0 h_c + sheme_c 11050 or 13569
UPP3MT: 2.0 amet_c + uppg3_c --> 2.0 ahcys_c + dscl_c + h_c 11050

SHCHD: dscl_c + nadp_c --> h_c + nadph_c + scl_c 13569
SHCHD2: dscl_c + nad_c --> h_c + nadh_c + scl_c 11050 or 13569
SHCHF: fe2_c + scl_c --> 3.0 h_c + sheme_c 11050 or 13569

CPPPGO: cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c 14197

PPPGOm: 3.0 o2_m + 2.0 pppg9_m --> 6.0 h2o_m + 2.0 ppp9_m 11867

16275 no reactions

HEMEOSm: frdp_m + h2o_m + pheme_m --> hemeO_m + ppi_m 11103

HEMEASm: 2.0 h_m + hemeO_m + 2.0 nadph_m + 2.0 o2_m --> 3.0 h2o_m + hemeA_m + 2.0 nadp_m 10228 

In [134]:
for r in sorted(model.genes.get_by_id('12731').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('co2_r').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

AMAOTrm: 8aonn_m + amet_m <=> amob_m + dann_m 12731
DBTSm: atp_m + co2_m + dann_m <=> adp_m + dtbt_m + 3.0 h_m + pi_m 12731

C3STDH1Pr: 4mzym_int1_r + nadp_r --> 4mzym_int2_r + co2_r + h_r + nadph_r 13724 or 8835
C3STDH1r: 4mzym_int1_r + nad_r --> 4mzym_int2_r + co2_r + h_r + nadh_r 13724 or 8835
C3STDH2er: nad_r + zym_int1_r --> co2_r + h_r + nadh_r + zym_int2_r 13724 or 8835
CO2ter: co2_c <=> co2_r 
FAS240COAer: 9.0 h_r + 3.0 malcoa_r + 6.0 nadph_r + stcoa_r --> 3.0 co2_r + 3.0 coa_r + 3.0 h2o_r + 6.0 nadp_r + ttccoa_r (10677 and 11343 and 16241 and 16695) or (10677 and 16241 and 16655 and 16695)
FAS260COAer: 3.0 h_r + malcoa_r + 2.0 nadph_r + ttccoa_r --> co2_r + coa_r + h2o_r + hexccoa_r + 2.0 nadp_r 10677 and 11343 and 16241 and 16695
PSERDer_RT: h_r + 0.01 ps_RT_r --> co2_r + 0.01 pe_RT_r 14554
SERPTer: h_r + pmtcoa_r + ser__L_r --> 3dsphgn_r + co2_r + coa_r 10303 and 9394 and 9425


In [135]:
# 8677
r = sce.reactions.get_by_id('5AOPtm').copy()
model.add_reactions([r])
# 11050 is methyltransferase, UPP3MT is correct, remove it from SHCHD2
# 13569 MET8 this step is NADH, remove SHCHD
# SHCHF is by MET8 not MET1
model.reactions.get_by_id('SHCHD2').gene_reaction_rule = '13569'
model.reactions.get_by_id('SHCHF').gene_reaction_rule = '13569'
model.remove_reactions(['SHCHD'], remove_orphans=True)
r = sce.reactions.get_by_id('PPPG9tm').copy()
model.add_reactions([r])
r = sce.reactions.get_by_id('FCLTm').copy()
r.gene_reaction_rule = '16275'
model.add_reactions([r])

In [136]:
with model:
    sol = model.optimize()
    print(sol.objective_value)
    for k, v in sol.shadow_prices.items():
        if v and k in [m.id for m in model.reactions.BIOMASS_RT.metabolites]:
            print(k, v)

0.13657749916638628
adp_c -0.4595342375266898
atp_c -0.4708528148056722
amp_c -0.4482156602477074
cmp_c -0.05885660185070918
glu__L_c -0.028673729106759027
gmp_c -0.4746256738986702
ser__L_c -0.15996922554295429
cys__L_c -0.13733207098498712
ala__L_c -0.09658519278064931
ump_c -0.04753802457172679
gln__L_c -0.039992306385741416
dgmp_c -0.46330709661968755
pe_RT_r -264.0141153222685
his__L_c -0.25806356196080005
dtmp_c -0.18487009555671297
trp__L_c -0.2278806892168488
met__L_c -0.126013493706006
pro__L_c -0.01735515182777614
phe__L_c -0.02263715455796597
tyr__L_c -0.033955731836948666
asp__L_c -1.3322676295501878e-15
ile__L_c -0.012073149097582753
dcmp_c -0.03621944729274151
mannan_r -0.011318577278982389
damp_c -0.43689708296872476
arg__L_c -0.10488548278524032
asn__L_c -0.02263715455796611
glycogen_c -0.022637154557962183
13BDglcn_c -0.022637154557962183
thr__L_c -4.440892098500626e-16
lys__L_c -0.02716458546956224
val__L_c -0.015846008190576882
ergst_r 1.967168731087132
tre_c -0.0339

In [137]:
model.summary()

In [138]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
model

1122
2115
1976


Name,R. toruloides
Memory address,0x010227fdbe0
Number of metabolites,1976
Number of reactions,2115
Number of groups,0
Objective expression,1.0*BIOMASS_RT - 1.0*BIOMASS_RT_reverse_2b3e0
Compartments,"c, x, m, e, r, v, n, g, d"


In [139]:
for x in sorted(model.genes, key=lambda x: x.id):
    if not x.reactions:
        print(x)
print()
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)

16454
2542_AT1

adp_x


In [140]:
cobra.manipulation.remove_genes(model, [x for x in model.genes if not x.reactions])
model.remove_metabolites([x for x in model.metabolites if not x.reactions])

In [141]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
print(len(set([m.id.rsplit('_',1)[0] for m in model.metabolites])))
print(len(model.compartments))
model

1120
2115
1975
1236
9


Name,R. toruloides
Memory address,0x010227fdbe0
Number of metabolites,1975
Number of reactions,2115
Number of groups,0
Objective expression,1.0*BIOMASS_RT - 1.0*BIOMASS_RT_reverse_2b3e0
Compartments,"c, x, m, e, r, v, n, g, d"


In [142]:
cobra.io.save_json_model(model, "IFO0880_GPR_2a.json")

In [143]:
model_old = cobra.io.load_json_model("../1.Manual_curation/IFO0880_GPR_1f.json")
model_new = cobra.io.load_json_model("IFO0880_GPR_2a.json")

In [144]:
print('Removed reactions\n')
for r in sorted(model_old.reactions, key=lambda x: x.id):
    if r not in model_new.reactions:
        print(r)

Removed reactions

2AMADPTm: L2aadp_c + akg_m <=> L2aadp_m + akg_c
2DHPFALDL: 2dhp_c <=> 3mob_c + fald_c
4ABUTtmi: 4abut_c + h_c --> 4abut_m + h_m
5HOXINDACTO2OX: 5hoxindact_c + h2o_c + o2_c --> 5hoxindoa_c + h2o2_c + h_c
AASAD2: L2aadp_c + atp_c + h_c + nadh_c --> L2aadp6sa_c + amp_c + nad_c + ppi_c
AASAD3: L2aadp6sa_c + h2o_c + nad_c --> L2aadp_c + 2.0 h_c + nadh_c
ACCOAtm: accoa_c + coa_m <=> accoa_m + coa_c
ACCOAtx: accoa_c + coa_x <=> accoa_x + coa_c
AGMT: agm_c + h2o_c --> ptrc_c + urea_c
AKGICITtm: akg_c + icit_m <=> akg_m + icit_c
ALR2x: h_c + mthgxl_c + nadh_c --> acetol_c + nad_c
ARGNm: arg__L_m + h2o_m --> orn_m + urea_m
ASNtx: asn__L_c + h_c <=> asn__L_x + h_x
ASPGLU2m: asp__L_m + glu__L_c --> asp__L_c + glu__L_m
ASPt2m: asp__L_c + h_c <=> asp__L_m + h_m
ASPtx: asp__L_x + h_x <=> asp__L_c + h_c
BALAVECSEC: 3.0 ala_B_c + atp_c + h2o_c --> adp_c + 3.0 ala_B_e + h_c + pi_c
CITtbm: cit_c + pep_m <=> cit_m + pep_c
CMPtm: cmp_c <-- cmp_m
CRNCARtm: acrn_c + crn_m --> acrn_m + crn_

In [145]:
print('Updated reactions\n')
for r in sorted(model_old.reactions, key=lambda x: x.id):
    if r in model_new.reactions:
        r2 = model_new.reactions.get_by_id(r.id)
        if (r.name == r2.name and r.reaction == r2.reaction and r.gene_reaction_rule == r2.gene_reaction_rule and
            r.lower_bound == r2.lower_bound and r.upper_bound == r2.upper_bound):
            pass
        else:
            print('Old', r, r.gene_reaction_rule)
            print('New', r2, r2.gene_reaction_rule)
            print()

Updated reactions

Old 3HKYNAKGAT: akg_c + hLkynr_c --> 42A3HP24DB_c + glu__L_c 12407 or 14908 or 15839 or 8540
New 3HKYNAKGAT: akg_c + hLkynr_c --> 42A3HP24DB_c + glu__L_c 14908 or 15839 or 8540

Old 3SALACBOXL: 3sala_c + h_c --> co2_c + hyptaur_c 10722 or 9434 or 9435
New 3SALACBOXL: 3sala_c + h_c --> co2_c + hyptaur_c 10722 or 9434 or 9435

Old 5AOPt2: 5aop_e + h_e --> 5aop_c + h_c 11269 or 13423
New 5AOPt2: 5aop_e + h_e --> 5aop_c + h_c 11269

Old 5HOXINDACTOX: 5hoxindact_c + h2o_c + nad_c --> 5hoxindoa_c + 2.0 h_c + nadh_c 12042 or 13426 or 15814 or 16323
New 5HOXINDACTOX: 5hoxindact_c + h2o_c + nad_c --> 5hoxindoa_c + 2.0 h_c + nadh_c 12042 or 13426 or 16323

Old AASAD1: L2aadp_c + atp_c + h_c + nadph_c --> L2aadp6sa_c + amp_c + nadp_c + ppi_c 9495 or (10220 and 9495)
New AASAD1: L2aadp_c + atp_c + h_c + nadph_c --> L2aadp6sa_c + amp_c + nadp_c + ppi_c 10220 and 9495

Old AATA: 2oxoadp_c + glu__L_c <=> L2aadp_c + akg_c 12407 or 14908 or 15839
New AATA: 2oxoadp_c + glu__L_c <=> L2

In [146]:
print('Added reactions\n')
for r in sorted(model_new.reactions, key=lambda x: x.id):
    if r not in model_old.reactions:
        print(r)

Added reactions

2AMACHYD: 2amac_c + h2o_c --> nh4_c + pyr_c
2OBUTt: 2obut_e + h_e <=> 2obut_c + h_c
2OBUTtm: 2obut_c <=> 2obut_m
34HPPOR: 34hpp_c + o2_c --> co2_c + hgentis_c
34HPPYRDC: 34hpp_c + h_c --> 4hoxpacd_c + co2_c
3MOBtm: 3mob_c <=> 3mob_m
3MOPtm: 3mop_c <=> 3mop_m
3SALAOX: 2.0 3sala_c + 2.0 h_c + o2_c --> 2.0 Lcyst_c
3SPYRSP: 3snpyr_c + h2o_c --> h_c + pyr_c + so3_c
3SPYRSPm: 3snpyr_m + h2o_m --> h_m + pyr_m + so3_m
4MOPtm: 4mop_c <=> 4mop_m
5AOPtm: 5aop_c <=> 5aop_m
AASAD3m: L2aadp6sa_m + h2o_m + nad_m --> L2aadp_m + 2.0 h_m + nadh_m
ACALDtm: acald_m <=> acald_c
ACRNtp: acrn_x --> acrn_c
ADPter: adp_r <=> adp_c
AHCYStr: ahcys_c <=> ahcys_r
AM6SAD: am6sa_c + h2o_c + nad_c --> amuco_c + 2.0 h_c + nadh_c
AMCOXO: amuco_c + h2o_c + h_c + nadph_c --> 2oxoadp_c + nadp_c + nh4_c
AMETr: amet_c <=> amet_r
AMPter: amp_c <=> amp_r
ARGt7m: arg__L_c --> arg__L_m
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
ATPter: atp_c <=> atp_r
BIOMASS_RT: 1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_